In [ ]:
# -*- coding: utf-8 -*-
"""
Pairs trading BDR/ADR com Filtro de Kalman, custos e grid search.

Requisitos: pandas, numpy
pip install pandas numpy
"""

from __future__ import annotations
import math
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple, Dict, List, Optional


# ============================
# Utilidades gerais
# ============================

def to_datetime_index(df: pd.DataFrame, tz: Optional[str] = None) -> pd.DataFrame:
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
    df = df.dropna(subset=["timestamp"]).sort_values("timestamp")
    if tz:
        df["timestamp"] = df["timestamp"].dt.tz_convert(tz)
    df = df.set_index("timestamp")
    return df


def align_series(
    bdr: pd.DataFrame,
    adr: pd.DataFrame,
    price_col: str = "close",
    how: str = "inner",
    ffill: bool = True,
) -> Tuple[pd.Series, pd.Series]:
    """Alinha por índice temporal e retorna duas séries de preço."""
    b = bdr[[price_col]].rename(columns={price_col: "bdr"}).copy()
    a = adr[[price_col]].rename(columns={price_col: "adr"}).copy()
    df = b.join(a, how=how)
    if ffill:
        df = df.ffill().dropna()
    return df["bdr"], df["adr"]


def log_prices(bdr: pd.Series, adr: pd.Series) -> Tuple[pd.Series, pd.Series]:
    return np.log(bdr.astype(float)), np.log(adr.astype(float))


def realized_bar_seconds(index: pd.DatetimeIndex) -> float:
    """Duração média (em segundos) por barra, a partir do índice."""
    if len(index) < 2:
        return 0.0
    dt = (index[1:] - index[:-1]).total_seconds()
    return float(np.median(dt))


def max_drawdown(series: pd.Series) -> float:
    """Retorna MDD em unidades da própria série (mesma moeda/unidade do equity)."""
    roll_max = series.cummax()
    drawdown = series - roll_max
    return float(drawdown.min())  # negativo


# ============================
# Filtro de Kalman (modelo de par)
# ============================

@dataclass
class KFConfig:
    # ruído de processo (Q) para alfa e beta (passeio aleatório)
    q_alpha: float = 1e-7
    q_beta: float = 1e-6
    # ruído de medição (R) inicial; pode ser atualizado via EWMA dos resíduos
    r_init: float = 1e-3
    # fator de esquecimento (EWMA) para R adaptativo
    r_ewma_lambda: float = 0.01
    # inicialização de P (covariância do estado)
    p0_alpha: float = 1.0
    p0_beta: float = 1.0


class KalmanPair:
    """
    Estado: theta_t = [alpha_t, beta_t]^T
    Medição: y_t = alpha_t + beta_t * x_t + v_t,  v_t ~ N(0, R_t)
    Evolução: theta_t = theta_{t-1} + w_t,        w_t ~ N(0, Q)
    Q = diag(q_alpha, q_beta); R adaptativo por EWMA dos resíduos.
    """

    def __init__(self, cfg: KFConfig):
        self.cfg = cfg
        self._fitted = False

    def fit_filter(self, y: pd.Series, x: pd.Series) -> pd.DataFrame:
        yv = y.astype(float).values
        xv = x.astype(float).values
        n = len(yv)
        # Estado e covariâncias
        theta = np.zeros((2, 1))  # [alpha, beta]
        P = np.diag([self.cfg.p0_alpha, self.cfg.p0_beta]).astype(float)
        Q = np.diag([self.cfg.q_alpha, self.cfg.q_beta]).astype(float)
        R = float(self.cfg.r_init)

        # buffers
        alpha_f = np.zeros(n)
        beta_f = np.zeros(n)
        resid = np.zeros(n)
        resid_var = np.zeros(n)
        zscore = np.zeros(n)
        p_alpha = np.zeros(n)
        p_beta = np.zeros(n)

        # EWMA para variância do resíduo
        ewma_var = R

        for t in range(n):
            H = np.array([[1.0, xv[t]]])  # 1x2
            # --- predição
            theta_pred = theta  # passeio aleatório => F = I
            P_pred = P + Q

            # --- inovação
            y_pred = float(H @ theta_pred)
            e_t = yv[t] - y_pred
            S_t = float(H @ P_pred @ H.T + R)  # variância da inovação

            # --- ganho de Kalman
            K_t = (P_pred @ H.T) / S_t  # 2x1

            # --- atualização
            theta = theta_pred + K_t * e_t
            P = (np.eye(2) - K_t @ H) @ P_pred

            # atualizar R (adaptativo) via EWMA da variância do resíduo
            ewma_var = (1 - self.cfg.r_ewma_lambda) * ewma_var + self.cfg.r_ewma_lambda * (e_t ** 2)
            R = float(max(1e-12, ewma_var))  # evitar zero

            # guardas
            alpha_f[t] = theta[0, 0]
            beta_f[t] = theta[1, 0]
            resid[t] = e_t
            resid_var[t] = S_t
            # z-score usando desvio da inovação (mais conservador)
            zscore[t] = e_t / (math.sqrt(S_t) + 1e-12)
            p_alpha[t] = P[0, 0]
            p_beta[t] = P[1, 1]

        out = pd.DataFrame(
            {
                "alpha": alpha_f,
                "beta": beta_f,
                "resid": resid,
                "resid_var": resid_var,
                "zscore": zscore,
                "p_alpha": p_alpha,
                "p_beta": p_beta,
            },
            index=y.index,
        )
        self._fitted = True
        self.results_ = out
        return out


# ============================
# Baseline OLS estático
# ============================

def ols_hedge(y: pd.Series, x: pd.Series) -> Tuple[float, float]:
    """Retorna (alpha, beta) via OLS: y = alpha + beta * x + e."""
    X = np.vstack([np.ones_like(x.values), x.values]).T  # [1, x]
    beta_hat = np.linalg.lstsq(X, y.values, rcond=None)[0]
    alpha, beta = float(beta_hat[0]), float(beta_hat[1])
    return alpha, beta


# ============================
# Estratégia e backtest
# ============================

@dataclass
class Costs:
    cost_bdr_bps: float = 10.0    # custo (por lado) no BDR
    cost_syn_bps: float = 2.0     # custo (por lado) no ADR "sintético"
    borrow_bps_day: float = 0.0   # bps/dia sobre o valor de mercado do perna short


@dataclass
class StratParams:
    z_entry: float = 1.5
    z_stop: float = 3.5


@dataclass
class BacktestResult:
    pair: str
    params: StratParams
    costs: Costs
    bars: int
    trades: int
    winrate: float
    expectancy_per_trade: float
    median_hold_bars: float
    sharpe_hourly_annualized: float
    mdd: float
    pnl_total: float
    costs_total: float


class PairStrategy:
    """
    Sinal:
      - Abrir posição quando |z| >= z_entry.
        * z > 0: short spread => vender 1x BDR, comprar beta_t * ADR
        * z < 0: long  spread => comprar 1x BDR, vender beta_t * ADR
      - Fechar no cruzamento de z por 0 (take-profit) OU se |z| sobe até z_stop contra a posição.
    PnL:
      - dPnL ≈ pos * Δ(resid)  (com pos = +1 para long spread, -1 para short spread)
      - Custos de entrada/saída em bps por perna e *borrow* por perna short pró-rata no tempo.
    """

    def __init__(self, y_log: pd.Series, x_log: pd.Series, kf: pd.DataFrame):
        self.y_log = y_log
        self.x_log = x_log
        self.kf = kf
        # Spread modelado: s_t = y - alpha - beta*x
        self.spread = y_log - kf["alpha"] - kf["beta"] * x_log
        self.z = kf["zscore"]

    def run(self, params: StratParams, costs: Costs) -> BacktestResult:
        idx = self.z.index
        n = len(idx)
        if n < 3:
            raise ValueError("Série muito curta para backtest.")

        # Estado de posição: 0 = flat, +1 = long spread, -1 = short spread
        pos = 0
        entry_z = params.z_entry
        stop_z = params.z_stop

        # buffers
        pos_series = np.zeros(n, dtype=int)
        dPnL = np.zeros(n)
        dCost = np.zeros(n)
        open_i = None  # índice da barra de abertura para medir duração

        # capital de referência: notional inicial do par (1x y + |beta0| x), em unidades de preço log ≈ escapar; 
        # Para retorno, usamos diretamente PnL da spread (unidade: "log-preço"), e anualizamos por hora.
        # Para imprimir PnL "total", mantemos a soma absoluta (mesma unidade do spread em log). Opcionalmente, você pode escalar por preço nível.

        # Para custos monetizáveis, estimamos notional por barra a partir dos níveis em preço (não log),
        # então precisamos dos níveis originais:
        # Reconstrói níveis aproximados (exp dos logs)
        y_lvl = np.exp(self.y_log.values)
        x_lvl = np.exp(self.x_log.values)
        beta = self.kf["beta"].values

        # segundos por barra e fator de anualização por hora
        sec_bar = realized_bar_seconds(idx)
        hours_bar = max(1e-9, sec_bar / 3600.0)
        ann_factor = math.sqrt(24.0 * 252.0 / max(1e-9, hours_bar))  # Sharpe anualizado por hora

        trades = []
        trade_pnls = []
        holds = []

        for t in range(1, n):
            z_prev, z_now = float(self.z.iloc[t - 1]), float(self.z.iloc[t])
            s_prev, s_now = float(self.spread.iloc[t - 1]), float(self.spread.iloc[t])

            # sinais de entrada
            if pos == 0:
                if z_now >= entry_z:
                    pos = -1  # short spread
                    open_i = t
                    # custos de abrir (duas pernas, por lado)
                    notional_b = y_lvl[t]
                    notional_a = abs(beta[t]) * x_lvl[t]
                    dCost[t] -= (costs.cost_bdr_bps / 1e4) * notional_b
                    dCost[t] -= (costs.cost_syn_bps / 1e4) * notional_a

                elif z_now <= -entry_z:
                    pos = +1  # long spread
                    open_i = t
                    notional_b = y_lvl[t]
                    notional_a = abs(beta[t]) * x_lvl[t]
                    dCost[t] -= (costs.cost_bdr_bps / 1e4) * notional_b
                    dCost[t] -= (costs.cost_syn_bps / 1e4) * notional_a

            else:
                # stop contra a posição
                if (pos < 0 and z_now >= stop_z) or (pos > 0 and z_now <= -stop_z):
                    # fechar
                    notional_b = y_lvl[t]
                    notional_a = abs(beta[t]) * x_lvl[t]
                    dCost[t] -= (costs.cost_bdr_bps / 1e4) * notional_b
                    dCost[t] -= (costs.cost_syn_bps / 1e4) * notional_a
                    # encerra trade
                    pos = 0
                    if open_i is not None:
                        holds.append(t - open_i)
                        open_i = None

                # take-profit no cruzamento de zero
                elif (pos < 0 and z_prev > 0.0 and z_now <= 0.0) or (pos > 0 and z_prev < 0.0 and z_now >= 0.0):
                    notional_b = y_lvl[t]
                    notional_a = abs(beta[t]) * x_lvl[t]
                    dCost[t] -= (costs.cost_bdr_bps / 1e4) * notional_b
                    dCost[t] -= (costs.cost_syn_bps / 1e4) * notional_a
                    pos = 0
                    if open_i is not None:
                        holds.append(t - open_i)
                        open_i = None

                # custo de borrow por barra (se houver perna short)
                if pos != 0 and costs.borrow_bps_day > 0:
                    # Identifica pernas *short* para +1 (long spread) e -1 (short spread)
                    # long spread: comprar BDR, vender beta*ADR  => short ADR
                    # short spread: vender BDR, comprar beta*ADR => short BDR
                    if pos > 0:  # short na perna ADR
                        short_notional = abs(beta[t]) * x_lvl[t]
                    else:        # short na perna BDR
                        short_notional = y_lvl[t]
                    dCost[t] -= (costs.borrow_bps_day / 1e4) * short_notional * (hours_bar / 24.0)

            # PnL da mudança do spread
            dPnL[t] = pos * (s_now - s_prev)
            pos_series[t] = pos

            # marca trades no take/stop (quando pos transiciona)
            if pos_series[t] == 0 and pos_series[t - 1] != 0:
                # consolidar PnL do trade recente
                # (aqui, para simplicidade, atribuimos ao último bar)
                pass

        # Curva
        pnl = pd.Series(dPnL, index=idx).cumsum()
        costs_curve = pd.Series(dCost, index=idx).cumsum()
        equity = pnl + costs_curve

        # Agregações de trades (heurística: delimitar por transições de pos)
        in_pos = False
        last_eq = 0.0
        eq_series = equity.values
        for t in range(1, n):
            if not in_pos and pos_series[t] != 0:
                in_pos = True
                last_eq = eq_series[t - 1]
                trade_start = t
            elif in_pos and pos_series[t] == 0:
                in_pos = False
                trade_pnls.append(eq_series[t] - last_eq)
                trades.append((trade_start, t))

        trades_count = len(trades)
        winrate = float(np.mean(np.array(trade_pnls) > 0.0)) if trades_count > 0 else 0.0
        expectancy = float(np.mean(trade_pnls)) if trades_count > 0 else 0.0
        median_hold = float(np.median(holds)) if holds else 0.0

        # Retornos por barra para Sharpe
        # retorno ≈ Δequity; como não escalamos por capital fixo, Sharpe é relativo à unidade do spread;
        # o fator de anualização por hora coloca todas as comparações no mesmo ritmo temporal.
        rets = np.diff(equity.values, prepend=equity.values[0])
        vol = float(np.std(rets, ddof=1))
        mean = float(np.mean(rets))
        sharpe_hourly = ann_factor * (mean / (vol + 1e-12))

        res = BacktestResult(
            pair="",
            params=params,
            costs=costs,
            bars=n,
            trades=trades_count,
            winrate=winrate,
            expectancy_per_trade=expectancy,
            median_hold_bars=median_hold,
            sharpe_hourly_annualized=sharpe_hourly,
            mdd=max_drawdown(equity),
            pnl_total=float(equity.iloc[-1]),
            costs_total=float(costs_curve.iloc[-1]),
        )
        return res


# ============================
# Avaliações e tabelas
# ============================

@dataclass
class PairRunSummary:
    bdr: str
    adr: str
    overlap_obs: int
    mae_teorico: float
    mae_kf: float
    sharpe_base: float
    pnl_base: float
    trades_base: int
    sharpe_kf: float
    pnl_kf: float
    trades_kf: int
    mdd_kf: float


def compare_base_vs_kf(
    y_log: pd.Series,
    x_log: pd.Series,
    kf_df: pd.DataFrame,
    hours_bar: float,
) -> Tuple[float, float, float, float]:
    """Retorna (sharpe_base, pnl_base, sharpe_kf, pnl_kf) em modelo simples sem custos, para comparação rápida."""
    # Base OLS
    a0, b0 = ols_hedge(y_log, x_log)
    spread_base = y_log - a0 - b0 * x_log
    ds = spread_base.diff().fillna(0.0).values
    ann = math.sqrt(24.0 * 252.0 / max(1e-9, hours_bar))
    sharpe_base = ann * (np.mean(ds) / (np.std(ds, ddof=1) + 1e-12))
    pnl_base = float(np.nansum(ds))

    # KF (sem execução/stop/custos — só como "série")
    spread_kf = y_log - kf_df["alpha"] - kf_df["beta"] * x_log
    dk = spread_kf.diff().fillna(0.0).values
    sharpe_kf = ann * (np.mean(dk) / (np.std(dk, ddof=1) + 1e-12))
    pnl_kf = float(np.nansum(dk))
    return sharpe_base, pnl_base, sharpe_kf, pnl_kf


def mae_residual(y_log: pd.Series, x_log: pd.Series, alpha: np.ndarray, beta: np.ndarray) -> float:
    resid = y_log.values - alpha - beta * x_log.values
    return float(np.mean(np.abs(resid)))


def grid_search_for_pair(
    bdr_name: str,
    adr_name: str,
    bdr_df: pd.DataFrame,
    adr_df: pd.DataFrame,
    kf_cfg: KFConfig,
    Z_ENTRIES: List[float],
    Z_STOPS: List[float],
    COSTS_BDR_BPS: List[float],
    COSTS_SYN_BPS: List[float],
    BORROW_BPS_DAY: List[float],
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Retorna:
      - topo_kf: resumo curto tipo "Top N por Sharpe (KF)"
      - comparativo: base × kf por par
      - grid: tabela longa de backtests (uma linha por combinação)
    """
    bdr_df = to_datetime_index(bdr_df)
    adr_df = to_datetime_index(adr_df)
    y_lvl, x_lvl = align_series(bdr_df, adr_df)
    overlap = len(y_lvl)

    # logs
    y_log, x_log = log_prices(y_lvl, x_lvl)

    # Filtro de Kalman
    kf = KalmanPair(kf_cfg).fit_filter(y_log, x_log)

    # OLS "teórico" (estático) e MAE
    a0, b0 = ols_hedge(y_log, x_log)
    mae_theoretical = mae_residual(y_log, x_log, alpha=a0, beta=b0)
    mae_kf = mae_residual(y_log, x_log, alpha=kf["alpha"].values, beta=kf["beta"].values)

    # Infos rápidas para o comparativo
    sec_bar = realized_bar_seconds(y_log.index)
    hours_bar = max(1e-9, sec_bar / 3600.0)
    sharpe_base, pnl_base, sharpe_kf_naive, pnl_kf_naive = compare_base_vs_kf(y_log, x_log, kf, hours_bar)

    # Correlação de log-preços e indicador de incerteza relativa do beta
    corr_log = float(pd.Series(y_log).corr(pd.Series(x_log)))
    # median_ratio_diag: mediana de sqrt(Var(beta_t)) / |beta_t|
    beta = kf["beta"].values
    p_beta = kf["p_beta"].values
    ratio_diag = np.median(np.sqrt(np.maximum(0.0, p_beta)) / (np.abs(beta) + 1e-12))
    median_ratio_diag = float(ratio_diag)

    # Backtest com execução/custos por grid
    rows = []
    top_rows = []  # para "Top N por Sharpe (KF)"

    for z_in in Z_ENTRIES:
        for z_st in Z_STOPS:
            params = StratParams(z_entry=z_in, z_stop=z_st)
            for cb in COSTS_BDR_BPS:
                for cs in COSTS_SYN_BPS:
                    for bw in BORROW_BPS_DAY:
                        strat = PairStrategy(y_log, x_log, kf)
                        res = strat.run(params, Costs(cb, cs, bw))
                        rows.append(
                            dict(
                                pair=f"{bdr_name}_{adr_name}",
                                z_entry=z_in,
                                z_stop=z_st,
                                cost_bdr_bps=cb,
                                cost_syn_bps=cs,
                                borrow_bps_day=bw,
                                bars=res.bars,
                                trades=res.trades,
                                winrate=res.winrate,
                                expectancy_per_trade=res.expectancy_per_trade,
                                median_hold_bars=res.median_hold_bars,
                                sharpe_hourly_annualized=res.sharpe_hourly_annualized,
                                mdd=res.mdd,
                                pnl_total=res.pnl_total,
                                costs_total=res.costs_total,
                            )
                        )

                        # coletar algumas combinações "padrão" (ex.: custos base) para o topo
                        if abs(cs - COSTS_SYN_BPS[0]) < 1e-9 and abs(cb - COSTS_BDR_BPS[0]) < 1e-9 and abs(bw - BORROW_BPS_DAY[0]) < 1e-9:
                            top_rows.append(
                                dict(
                                    bdr=bdr_name,
                                    adr=adr_name,
                                    overlap_obs=overlap,
                                    mae_teorico=mae_theoretical,
                                    mae_kf=mae_kf,
                                    sharpe_base=sharpe_base,
                                    pnl_base=pnl_base,
                                    trades_base=np.nan,  # baseline aqui é série, não execução
                                    sharpe_kf=res.sharpe_hourly_annualized,
                                    pnl_kf=res.pnl_total,
                                    trades_kf=res.trades,
                                    mdd_kf=res.mdd,
                                )
                            )

    grid_df = pd.DataFrame(rows).sort_values(["pair", "sharpe_hourly_annualized"], ascending=[True, False])
    top_df = pd.DataFrame(top_rows).sort_values("sharpe_kf", ascending=False)

    # comparativo curto (1 linha por par)
    comp_df = pd.DataFrame(
        [
            dict(
                bdr=bdr_name,
                adr=adr_name,
                sharpe_base=sharpe_base,
                pnl_base=pnl_base,
                trades_base=int(grid_df["trades"].median()) if len(grid_df) else 0,
                sharpe_kf=sharpe_kf_naive,  # nota: "naive série" (sem execução)
                pnl_kf=pnl_kf_naive,
                trades_kf=int(grid_df["trades"].median()) if len(grid_df) else 0,
                corr_log=corr_log,
                median_ratio_diag=median_ratio_diag,
            )
        ]
    )

    return top_df, comp_df, grid_df


# ============================
# Exemplo de orquestração
# ============================

if __name__ == "__main__":
    # Exemplo mínimo:
    # Troque 'data/AAPL34.csv' e 'data/AAPL.csv' pelos seus caminhos.
    # CSV esperado: timestamp,close
    try:
        aapl34 = pd.read_csv("data/AAPL34.csv")
        aapl_adr = pd.read_csv("data/AAPL.csv")
        tsla34 = pd.read_csv("data/TSLA34.csv")
        tsla_adr = pd.read_csv("data/TSLA.csv")
    except Exception:
        # Se não houver arquivos, crie séries dummy só para não quebrar
        rng = pd.date_range("2023-01-01", periods=1000, freq="H", tz="UTC")
        base = np.cumsum(np.random.randn(len(rng)) * 0.001) + 10
        aapl34 = pd.DataFrame({"timestamp": rng, "close": np.exp(base + 0.02*np.random.randn(len(rng)))})
        aapl_adr = pd.DataFrame({"timestamp": rng, "close": np.exp(base + 0.02*np.random.randn(len(rng)))})
        tsla34 = pd.DataFrame({"timestamp": rng, "close": np.exp(base + 0.03*np.random.randn(len(rng)))})
        tsla_adr = pd.DataFrame({"timestamp": rng, "close": np.exp(base + 0.03*np.random.randn(len(rng)))})

    KF_CFG = KFConfig(
        q_alpha=1e-7,
        q_beta=1e-6,
        r_init=1e-3,
        r_ewma_lambda=0.01,
        p0_alpha=1.0,
        p0_beta=1.0,
    )

    # Grids de parâmetros e custos (ajuste conforme seu universo)
    Z_ENTRIES = [1.5, 2.0, 2.5]
    Z_STOPS = [3.0, 3.5, 4.0]
    COSTS_BDR_BPS = [10.0, 20.0]   # por lado
    COSTS_SYN_BPS = [2.0, 5.0]     # por lado
    BORROW_BPS_DAY = [0.0, 20.0, 40.0]

    # Rode para um par
    top_aapl, comp_aapl, grid_aapl = grid_search_for_pair(
        "AAPL34",
        "AAPL",
        aapl34,
        aapl_adr,
        KF_CFG,
        Z_ENTRIES,
        Z_STOPS,
        COSTS_BDR_BPS,
        COSTS_SYN_BPS,
        BORROW_BPS_DAY,
    )

    top_tsla, comp_tsla, grid_tsla = grid_search_for_pair(
        "TSLA34",
        "TSLA",
        tsla34,
        tsla_adr,
        KF_CFG,
        Z_ENTRIES,
        Z_STOPS,
        COSTS_BDR_BPS,
        COSTS_SYN_BPS,
        BORROW_BPS_DAY,
    )

    # Consolidados
    print("\n== Top por Sharpe (KF) ==")
    print(pd.concat([top_aapl, top_tsla], ignore_index=True).head(10).to_string(index=False))

    print("\nResumo base x KF")
    print(pd.concat([comp_aapl, comp_tsla], ignore_index=True).to_string(index=False))

    print("\nGrid (amostra):")
    print(pd.concat([grid_aapl, grid_tsla], ignore_index=True).head(20).to_csv(index=False))


In [2]:
# %%
# -*- coding: utf-8 -*-
"""
ADR↔BDR — ETL + baseline teórico + Kalman (walk-forward) + backtest beta-neutral simplificado
(versão com ajustes de robustez do KF, sizing com incerteza de β e custos de execução)

Mudanças-chave (sem look-ahead):
- ρ calibrado só no primeiro TRAIN (evita forward-bias); α_t, β_t pelo Kalman com WFA + validação interna
- Penalização de var(Δβ) e |β-1| na escolha de (q_alpha, q_beta, r)
- Teórico dinâmico com β CLIPPED e LAGGED; gate_kf usa sd_beta e banda de β
- Sizing por risco: EWMA do spread + incerteza de β (lagged)
- Custos: borrow + FX basis + slippage & comissão via turnover
"""

from pathlib import Path
import sys, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint

# ============================ CONFIG & LOGGING ============================

CFG = {
    # dados
    "path_bdr_xlsx": "../data/Hist_BDRs.xlsx",
    "path_us_xlsx":  "../data/Hist_Origem_BDRs.xlsx",
    "path_fx_xlsx":  "../data/dolar.xlsx",
    "out_dir":       "../data/output",

    # pares (BDR -> ADR)
    "pairs": {
        "AAPL34": "AAPL",
        "MSFT34": "MSFT",
        "NVDC34": "NVDA",
        "AMZO34": "AMZN",
        "GOGL34": "GOOGL",
        "M1TA34": "META",
        "TSLA34": "TSLA",
        "TSMC34": "TSM",
        "AVGO34": "AVGO",
        "BABA34": "BABA",
        "JDCO34": "JD",
    },

    # Alinhamento temporal
    "shift_bdr_base_min": -30,   # -30min em todas as barras
    "shift_bdr_eod_min":  -60,   # -60min na última barra do dia
    "asof_tolerance": "31min",
    "min_overlap": 500,

    # Walk-Forward KF
    "wfa_train": 252,
    "wfa_test":  63,
    # grids mais conservadores (R maior ↓ reatividade)
    "q_alpha_grid": [1e-8, 1e-7, 1e-6, 1e-5],
    "q_beta_grid":  [1e-8, 1e-7, 1e-6, 1e-5],
    "r_grid":       [1e-4, 1e-3, 1e-2, 1e-1],
    "init_alpha": 0.0,
    "init_beta":  1.0,
    "init_p_alpha": 1.0,
    "init_p_beta":  1.0,

    # Regularização/penalização na seleção do KF
    "kf_beta_anchor": 1.0,
    "kf_w_anchor": 0.05,     # penaliza |β-1|
    "kf_w_smooth": 0.10,     # penaliza var(Δβ)

    # Clip/estabilidade do β e gate
    "kf_beta_clip": (0.3, 2.0),
    "kf_sd_beta_max": 0.25,  # em log-space
    "use_kf_stability_gate": True,

    # Z-Score robusto
    "winsor_pct": 0.01,
    "ewm_alpha": 0.06,  # ~half-life 16 barras

    # Gate de cointegração
    "coint_L": 252,
    "coint_pval": 0.05,

    # Sinais & sizing
    "z_enter": 2.0,
    "z_exit": 0.75,
    "z_stop": 3.0,
    "target_vol": 0.08,
    "max_leverage": 2.0,
    "uncertainty_weight": 1.0,  # peso da incerteza de β no sizing

    # Custos
    "slippage_bps": 3.0,
    "commission_bps": 3.0,
    "borrow_bps_pa": 400.0,  # aluguel short BDR
    "fx_basis_bps": 20.0,    # custo hedge FX aprox

    # Diversos
    "plot_top_n": None,
}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
log = logging.getLogger("pairs")

# ============================ I/O DE DADOS ============================

def _ptbr_to_float(series: pd.Series) -> pd.Series:
    try:
        return series.astype(float)
    except Exception:
        return pd.to_numeric(series, errors='coerce')

def load_bdr_sheets(path: Path) -> dict[str, pd.DataFrame]:
    raw = pd.read_excel(path, sheet_name=None)
    out = {}
    for sheet, df in raw.items():
        if 'Data' not in df.columns or 'Fechamento' not in df.columns:
            continue
        df = df.rename(columns={'Data':'datetime','Fechamento':'close_bdr','Volume Financeiro':'volume_bdr'})
        df['datetime']  = pd.to_datetime(df['datetime'], dayfirst=True)
        df['close_bdr'] = _ptbr_to_float(df['close_bdr'])
        if 'volume_bdr' in df.columns:
            df['volume_bdr'] = _ptbr_to_float(df['volume_bdr'])
        else:
            df['volume_bdr'] = np.nan
        df = (df[['datetime','close_bdr','volume_bdr']]
                .dropna(subset=['datetime','close_bdr'])
                .sort_values('datetime')
                .set_index('datetime'))
        out[sheet] = df
    return out

def load_us_sheets(path: Path) -> dict[str, pd.DataFrame]:
    raw = pd.read_excel(path, sheet_name=None)
    out = {}
    for sheet, df in raw.items():
        if 'Date' not in df.columns or 'Last Price' not in df.columns:
            continue
        df = df.rename(columns={'Date':'datetime','Last Price':'close_us','Volume':'volume_us'})
        df['datetime']  = pd.to_datetime(df['datetime'], dayfirst=True)
        df['close_us']  = _ptbr_to_float(df['close_us'])
        if 'volume_us' in df.columns:
            df['volume_us'] = _ptbr_to_float(df['volume_us'])
        else:
            df['volume_us'] = np.nan
        df = (df[['datetime','close_us','volume_us']]
                .dropna(subset=['datetime','close_us'])
                .sort_values('datetime')
                .set_index('datetime'))
        out[sheet] = df
    return out

def load_fx(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    if 'Date' not in df.columns or 'Mid Price' not in df.columns:
        raise ValueError("dolar.xlsx precisa conter colunas 'Date' e 'Mid Price'.")
    df = df.rename(columns={'Date':'datetime','Mid Price':'usdxbrl','Volume':'volume_fx'})
    df['datetime'] = pd.to_datetime(df['datetime'], dayfirst=True)
    df['usdxbrl']  = _ptbr_to_float(df['usdxbrl'])
    if 'volume_fx' in df.columns:
        df['volume_fx'] = _ptbr_to_float(df['volume_fx'])
    else:
        df['volume_fx'] = np.nan
    df = (df[['datetime','usdxbrl','volume_fx']]
            .dropna(subset=['datetime','usdxbrl'])
            .sort_values('datetime')
            .set_index('datetime'))
    return df

# ============================ ALINHAMENTO & TEÓRICO ============================

def shift_bdr_index_with_eod_rule(df_bdr: pd.DataFrame,
                                  base_min: int = -30,
                                  eod_min:  int = -60) -> pd.DataFrame:
    out = df_bdr.copy()
    idx = out.index
    idx_s = pd.Series(idx, index=idx)
    last_per_day = idx_s.groupby(idx_s.index.normalize()).transform('max')
    is_last = (idx_s == last_per_day)
    base_delta = pd.to_timedelta(base_min, unit='m')
    eod_delta  = pd.to_timedelta(eod_min,  unit='m')
    new_idx = pd.DatetimeIndex(idx + base_delta)
    new_idx = pd.DatetimeIndex(np.where(is_last.values,
                                        (idx + eod_delta).values,
                                        new_idx.values))
    out.index = new_idx
    return out.sort_index()

def nearest_join(left: pd.DataFrame, right: pd.DataFrame, tolerance: str, direction: str = "backward") -> pd.DataFrame:
    l = left.sort_index().reset_index().rename(columns={'index':'datetime'})
    r = right.sort_index().reset_index().rename(columns={'index':'datetime'})
    m = pd.merge_asof(l, r, on='datetime',
                      direction=direction,
                      tolerance=pd.to_timedelta(tolerance))
    m = m.set_index('datetime')
    return m

def theoretical_bdr_brl(px_adr_usd: pd.Series,
                        fx_brlusd: pd.Series,
                        rho: float) -> pd.Series:
    return (px_adr_usd * fx_brlusd * float(rho)).rename("theo_bdr_brl")

# ============================ RATIO (ρ) ============================

def calibrate_ratio(bdr_close: pd.Series, adr_usd: pd.Series, usdxbrl: pd.Series, method="ols"):
    denom = adr_usd * usdxbrl
    df = pd.DataFrame({'bdr': bdr_close, 'den': denom}).dropna()
    if df.empty: return np.nan, np.nan, 0
    ratio_series = df['bdr'] / df['den'].replace(0, np.nan)
    ratio_series = ratio_series.replace([np.inf, -np.inf], np.nan).dropna()
    if ratio_series.empty: return np.nan, np.nan, int(len(df))
    if method == "median":
        ratio = float(np.median(ratio_series.values))
    else:
        num = (df['den'] * df['bdr']).sum()
        den = (df['den'] ** 2).sum()
        ratio = float(num / den) if den != 0 else np.nan
    return ratio, float(ratio_series.std(ddof=1)), int(len(ratio_series))

def choose_ratio_with_fallback(df_pair: pd.DataFrame, ratio_ols: float):
    denom = df_pair['close_us'] * df_pair['usdxbrl']
    ratio_series = (df_pair['close_bdr'] / denom).replace([np.inf,-np.inf], np.nan).dropna()
    if ratio_series.empty:
        return ratio_ols, np.nan, 0, "ols"
    ratio_med = float(np.median(ratio_series))
    ratio_sd  = float(ratio_series.std(ddof=1)) if len(ratio_series) > 1 else 0.0
    if (ratio_med > 0) and (ratio_ols <= 0 or ratio_ols < ratio_med/100.0 or ratio_ols > ratio_med*100.0):
        return ratio_med, ratio_sd, len(ratio_series), "median_fallback"
    return ratio_ols, ratio_sd, len(ratio_series), "ols"

# ============================ ESTATÍSTICA ROBUSTA ============================

def robust_zscore(spread: pd.Series, ewm_alpha=0.06, winsor_pct=0.01) -> pd.Series:
    s = spread.copy()
    lo, hi = s.quantile([winsor_pct, 1-winsor_pct])
    s = s.clip(lo, hi)
    mu = s.ewm(alpha=ewm_alpha, adjust=False).mean().shift(1)
    dev = (s - mu).abs().ewm(alpha=ewm_alpha, adjust=False).mean().shift(1)
    eps = 1e-12
    return ((s - mu) / (dev + eps)).rename("z")

# ============================ COINTEGRAÇÃO (gate) ============================

def rolling_coint_gate(x: pd.Series, y: pd.Series, L=252, pval_th=0.05) -> pd.Series:
    gate = pd.Series(False, index=y.index)
    if len(y) < L: return gate
    for i in range(L, len(y)):
        _x = x.iloc[i-L:i].values
        _y = y.iloc[i-L:i].values
        try:
            _, pval, _ = coint(_y, _x)
            gate.iloc[i] = (pval < pval_th)
        except Exception:
            gate.iloc[i] = False
    return gate.rename("coint_on")

# ============================ KALMAN (manual, 2D) ============================

def kalman_alpha_beta(y: pd.Series, x: pd.Series,
                      q_alpha=1e-5, q_beta=1e-5, r=1e-4,
                      alpha0=0.0, beta0=1.0, p0_alpha=1.0, p0_beta=1.0):
    """
    Estado 2D: y_t = α_t + β_t x_t + e_t   (tudo em LOGs)
    Retorna DataFrame com alpha_t, beta_t, sd_alpha_t, sd_beta_t, spread_kf (inovação).
    """
    y = pd.Series(y).astype(float)
    x = pd.Series(x).astype(float)
    idx = y.index.intersection(x.index)
    y = y.loc[idx]; x = x.loc[idx]

    theta = np.array([alpha0, beta0], dtype=float)
    P = np.diag([p0_alpha, p0_beta])
    Q = np.diag([q_alpha, q_beta])
    R = np.array([[r]])

    alphas, betas, sd_a, sd_b, resid = [], [], [], [], []

    for t in range(len(idx)):
        H = np.array([[1.0, float(x.iloc[t])]])
        # Predição
        theta_pred = theta
        P_pred = P + Q
        # Inovação
        y_hat = float(H @ theta_pred)
        nu = float(y.iloc[t] - y_hat)
        S = float(H @ P_pred @ H.T + R)
        K = (P_pred @ H.T) / S
        # Atualização
        theta = theta_pred + (K.flatten() * nu)
        P = (np.eye(2) - K @ H) @ P_pred

        alphas.append(theta[0]); betas.append(theta[1])
        sd_a.append(np.sqrt(max(P[0,0], 0.0))); sd_b.append(np.sqrt(max(P[1,1], 0.0)))
        resid.append(nu)

    out = pd.DataFrame({
        "alpha_t": alphas,
        "beta_t":  betas,
        "sd_alpha_t": sd_a,
        "sd_beta_t":  sd_b,
        "spread_kf":  resid,
    }, index=idx)
    return out

def fit_kf_walkforward(x_log: pd.Series, y_log: pd.Series,
                       q_alpha_grid, q_beta_grid, r_grid,
                       train=252, test=63,
                       alpha0=0.0, beta0=1.0, p0_alpha=1.0, p0_beta=1.0,
                       w_anchor=0.05, beta_anchor=1.0, w_smooth=0.10):
    """
    WFA com validação interna:
      - Para cada janela TRAIN, escolhe (q_alpha, q_beta, r) que minimiza:
        OBJ = RMSE(residual) na *cauda* do TRAIN (últimos 20%)
              + w_anchor * mean((β-β_anchor)^2)
              + w_smooth * var(Δβ)
      - Reaplica KF em TRAIN+TEST com hiperparâmetros escolhidos e coleta α,β,sd's no TEST.
    """
    idx = y_log.index
    a_full = pd.Series(index=idx, dtype=float)
    b_full = pd.Series(index=idx, dtype=float)
    sa_full = pd.Series(index=idx, dtype=float)
    sb_full = pd.Series(index=idx, dtype=float)
    s_full = pd.Series(index=idx, dtype=float)

    i = train
    while i + test <= len(idx):
        tr = slice(idx[i-train], idx[i-1])
        te = slice(idx[i], idx[i+test-1])

        x_tr, y_tr = x_log.loc[tr], y_log.loc[tr]
        x_te, y_te = x_log.loc[te], y_log.loc[te]

        # parte de validação dentro do TRAIN (últimos 20%)
        cut = int(round(0.8 * len(x_tr)))
        x_val, y_val = x_tr.iloc[cut:], y_tr.iloc[cut:]

        best = None
        for qa in q_alpha_grid:
            for qb in q_beta_grid:
                for rr in r_grid:
                    kf_tr = kalman_alpha_beta(
                        y_tr, x_tr, q_alpha=qa, q_beta=qb, r=rr,
                        alpha0=alpha0, beta0=beta0, p0_alpha=p0_alpha, p0_beta=p0_beta
                    )
                    # métrica na cauda do TRAIN (proxy 'quase-OOS')
                    kf_tail = kf_tr.iloc[cut:].copy()
                    rmse_tail = float(np.sqrt(np.nanmean((kf_tail["spread_kf"].values)**2)))
                    # regularizações de estabilidade de β
                    beta_series = kf_tr["beta_t"].values
                    dbeta = np.diff(beta_series)
                    smooth_pen = np.nanvar(dbeta) if len(dbeta)>1 else 0.0
                    anchor_pen = np.nanmean((beta_series - beta_anchor)**2)
                    obj = rmse_tail + w_anchor*anchor_pen + w_smooth*smooth_pen

                    if (best is None) or (obj < best["obj"]):
                        best = {"qa": qa, "qb": qb, "rr": rr, "obj": obj}

        # Refit em TRAIN+TEST com os hiperparâmetros escolhidos
        x_block = pd.concat([x_tr, x_te], axis=0)
        y_block = pd.concat([y_tr, y_te], axis=0)
        kf_block = kalman_alpha_beta(
            y_block, x_block,
            q_alpha=best["qa"], q_beta=best["qb"], r=best["rr"],
            alpha0=alpha0, beta0=beta0, p0_alpha=p0_alpha, p0_beta=p0_beta
        )
        kk = kf_block.loc[te]
        a_full.loc[te]  = kk["alpha_t"]
        b_full.loc[te]  = kk["beta_t"]
        sa_full.loc[te] = kk["sd_alpha_t"]
        sb_full.loc[te] = kk["sd_beta_t"]
        s_full.loc[te]  = kk["spread_kf"]

        i += test

    return (a_full.rename("alpha_t"),
            b_full.rename("beta_t"),
            sa_full.rename("sd_alpha_t"),
            sb_full.rename("sd_beta_t"),
            s_full.rename("spread_kf"))

# ============================ MÉTRICAS/DIAGNÓSTICO ============================

def estimate_bars_per_day_from_index(index: pd.DatetimeIndex) -> int:
    if len(index) == 0: return 1
    counts = pd.Series(index.normalize()).value_counts()
    return int(np.median(counts.values))

def diagnostics_quick(df_pair: pd.DataFrame):
    x = np.log(df_pair['close_bdr'])
    y = np.log(df_pair['close_us'] * df_pair['usdxbrl'])
    valid = x.notna() & y.notna()
    corr = x[valid].corr(y[valid]) if valid.any() else np.nan
    ratio_inst = (df_pair['close_bdr'] / (df_pair['close_us'] * df_pair['usdxbrl'])).replace([np.inf,-np.inf], np.nan)
    med_ratio = float(ratio_inst.median())
    return {"corr_log": float(corr), "median_ratio": med_ratio, "n": int(valid.sum())}

# ============================ BACKTEST (next-bar, custos) ============================

def backtest_next_bar(df: pd.DataFrame,
                      z_col="z",
                      theo_col="bdr_teo",
                      z_entry=2.0, z_exit=0.75, z_stop=3.0,
                      slippage_bps=0.0, commission_bps=0.0,
                      borrow_bps_pa=400.0, fx_basis_bps=20.0,
                      use_risk_sizing=True, ewm_alpha=0.06,
                      target_vol=0.08, max_leverage=2.0,
                      gate: pd.Series | None = None,
                      sd_beta: pd.Series | None = None,
                      x_log: pd.Series | None = None,
                      uncertainty_weight: float = 1.0):
    """
    Backtest do spread S = BDR - Teórico (preço), execução NEXT-BAR.
    Sizing por risco usa EWMA(std(S)) e, opcionalmente, adiciona incerteza de β (lagged):

        var_eff_t = var_EWMA(S)_t  +  (theo_{t-1} * ln(X_t))^2 * Var(β_{t-1})

    onde X_t = teórico estático (ADR×FX×ρ) em preço; Var(β) ≈ sd_beta^2.

    Custos:
      - slippage + comissão por turnover |Δpos|*(pxB+pxT) * (bps/1e4)
      - aluguel short BDR (sobre notional da perna BDR)
      - FX basis (sobre notional da perna sintética)
    """
    df = df.copy().sort_index()

    S   = (df["close_bdr"] - df[theo_col]).rename("S")
    z   = df[z_col].copy()
    pxB = df["close_bdr"]
    pxT = df[theo_col]

    # Gate
    if gate is not None:
        gate = gate.reindex(df.index).fillna(False)
    else:
        gate = pd.Series(True, index=df.index)

    # var efetiva (EWMA do spread + incerteza beta)
    if use_risk_sizing:
        base_var = (S - S.ewm(alpha=ewm_alpha, adjust=False).mean()).pow(2).ewm(alpha=ewm_alpha, adjust=False).mean()
        extra = 0.0
        if (sd_beta is not None) and (x_log is not None):
            sd_b_lag = sd_beta.reindex(df.index).shift(1).fillna(method="ffill").fillna(0.0)
            lnX = x_log.reindex(df.index).fillna(method="ffill").fillna(0.0)  # log do teórico estático
            theo_lag = pxT.shift(1).fillna(method="ffill").fillna(pxT.iloc[0])
            extra = (theo_lag * lnX).pow(2) * (sd_b_lag.pow(2)) * float(uncertainty_weight)
        var_eff = (base_var + extra).replace(0, np.nan)
        vol_eff = np.sqrt(var_eff).shift(1)
        w = (target_vol / vol_eff).clip(upper=max_leverage).fillna(0.0)
    else:
        w = pd.Series(target_vol, index=df.index).clip(upper=max_leverage)

    # estado discreto
    state = pd.Series(0, index=df.index, dtype=int)
    pos = 0
    for t, zz in z.items():
        if not gate.loc[t]:
            pos = 0
        else:
            if pos == 0:
                if zz >  z_entry: pos = -1
                if zz < -z_entry: pos = +1
            else:
                if abs(zz) > z_stop:
                    pos = 0
                elif abs(zz) < z_exit:
                    pos = 0
        state.loc[t] = pos

    # posição efetiva (contínua)
    pos_w = (state * w).rename("pos")

    # Execução next-bar
    pos_exec = pos_w.shift(1).fillna(0.0)
    dS = S.diff().fillna(0.0)
    pnl_gross = (pos_exec * dS).rename("pnl_gross")

    # Turnover e custos de slippage/comissão (por mudança de posição)
    dpos = pos_exec.diff().abs().fillna(pos_exec.abs())
    bps_trading = (slippage_bps + commission_bps) / 1e4
    trade_cost = (dpos * (pxB.shift(1).fillna(pxB) + pxT.shift(1).fillna(pxT)) * bps_trading).rename("trade_cost")

    # Custos recorrentes (diários aprox)
    days = (df.index.to_series().diff().dt.days.fillna(0)).clip(lower=0)
    borrow_daily = (borrow_bps_pa/1e4) / 252.0
    fx_basis_daily = (fx_basis_bps/1e4) / 252.0

    borrow_cost = (pos_exec.clip(upper=0).abs() * pxB.shift(1) * borrow_daily * (days.replace(0,1))).rename("borrow_cost")
    fx_cost     = (pos_exec.abs() * pxT.shift(1) * fx_basis_daily * (days.replace(0,1))).rename("fx_cost")

    pnl_net = (pnl_gross - trade_cost - borrow_cost - fx_cost).rename("pnl_net")
    equity  = pnl_net.cumsum().rename("equity")

    # métricas
    bars_per_day = estimate_bars_per_day_from_index(df.index)
    ann_factor = np.sqrt(max(bars_per_day,1) * 252.0)
    ret = pnl_net
    std = float(ret.std(ddof=1))
    sharpe = (float(ret.mean()) / (std + 1e-12)) * ann_factor if std > 0 else np.nan
    roll_max = equity.cummax()
    mdd = float((equity - roll_max).min()) if len(equity) else np.nan

    # estatística de trades
    turns = (state.shift(1).fillna(0) != state).astype(int)
    trade_idx = turns[turns==1].index
    trades = int((state.abs().diff().fillna(state.abs()) > 0).sum())
    # winrate/expectancy aproximados por janelas entre flips
    wins = 0; losses = 0; pnl_trades = []
    if len(trade_idx) > 1:
        starts = trade_idx
        stops = list(starts[1:]) + [df.index[-1]]
        for s, e in zip(starts, stops):
            seg = equity.loc[s:e]
            if len(seg) >= 2:
                dp = float(seg.iloc[-1] - seg.iloc[0])
                pnl_trades.append(dp)
                if dp > 0: wins += 1
                elif dp < 0: losses += 1
    winrate = (wins / max(wins+losses,1)) if (wins+losses)>0 else np.nan
    expectancy = float(np.nanmedian(pnl_trades)) if len(pnl_trades)>0 else np.nan
    # holding mediano em barras
    hold_lengths = []
    if len(trade_idx) > 1:
        for s, e in zip(trade_idx, list(trade_idx[1:]) + [df.index[-1]]):
            hold_lengths.append(max(1, (df.index.get_loc(e) - df.index.get_loc(s))))
    med_hold = int(np.median(hold_lengths)) if hold_lengths else 0

    out = pd.concat([S, z, pos_w, pnl_gross, trade_cost, borrow_cost, fx_cost, pnl_net, equity], axis=1)
    summary = {
        "bars": len(df),
        "trades": trades,
        "winrate": winrate,
        "expectancy_per_trade": expectancy,
        "median_hold_bars": med_hold,
        "sharpe_hourly_annualized": sharpe,
        "mdd": mdd,
        "pnl_total": float(equity.iloc[-1]) if len(equity) else 0.0,
    }
    return out, summary

# ============================ PLOTS (opcional) ============================

def plot_prices(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    df[['close_bdr','bdr_teo']].dropna().plot(ax=plt.gca())
    plt.title(f"{name} — BDR vs Teórico")
    plt.xlabel("Tempo"); plt.ylabel("Preço (BRL)")
    fig.tight_layout(); fig.savefig(outdir / f"{name}_prices.png"); plt.close(fig)

def plot_spread(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    (df['close_bdr'] - df['bdr_teo']).dropna().plot(ax=plt.gca())
    plt.title(f"{name} — Spread (BDR − Teórico)")
    plt.xlabel("Tempo"); plt.ylabel("Spread (BRL)")
    fig.tight_layout(); fig.savefig(outdir / f"{name}_spread.png"); plt.close(fig)

def plot_zscore(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    df['z'].dropna().plot(ax=plt.gca())
    plt.title(f"{name} — Z-Score do Spread (robusto, shift(1))")
    plt.xlabel("Tempo"); plt.ylabel("Z")
    ax = plt.gca(); ax.axhline(0); ax.axhline(2); ax.axhline(-2)
    fig.tight_layout(); fig.savefig(outdir / f"{name}_zscore.png"); plt.close(fig)

# ============================ PIPELINE ============================

def run_pipeline_one_pair(bdr: str, adr: str,
                          bdr_map: dict, us_map: dict, fx_df: pd.DataFrame,
                          out_pairs_dir: Path, plots_dir: Path):
    # 1) Shift do BDR
    df_bdr = shift_bdr_index_with_eod_rule(
        bdr_map[bdr],
        base_min=CFG["shift_bdr_base_min"],
        eod_min=CFG["shift_bdr_eod_min"],
    )

    # 2) Nearest join BDR↔ADR↔FX (backward)
    tmp = nearest_join(
        df_bdr[['close_bdr','volume_bdr']],
        us_map[adr][['close_us','volume_us']],
        CFG["asof_tolerance"]
    )
    tmp = nearest_join(tmp, fx_df[['usdxbrl','volume_fx']], CFG["asof_tolerance"])
    tmp = tmp.dropna(subset=['close_bdr','close_us','usdxbrl']).sort_index()

    if len(tmp) < CFG["min_overlap"]:
        log.info(f"[{bdr}↔{adr}] overlap insuficiente: {len(tmp)}")
        return None

    # 3) ρ (ratio) usando apenas o primeiro TRAIN (evita look-ahead)
    n_train = min(CFG["wfa_train"], len(tmp)//3 or 1)
    tmp_head = tmp.iloc[:n_train]
    ratio_ols, ratio_sd_raw, n_ratio_raw = calibrate_ratio(
        tmp_head['close_bdr'], tmp_head['close_us'], tmp_head['usdxbrl'], method="ols"
    )
    ratio_used, ratio_sd, n_ratio, ratio_method = choose_ratio_with_fallback(tmp_head, ratio_ols)
    tmp["bdr_teo"] = theoretical_bdr_brl(tmp["close_us"], tmp["usdxbrl"], ratio_used)

    # 4) KF Walk-Forward (α_t, β_t) sobre LOGs (com validação interna e penalizações)
    x_log = np.log(tmp["bdr_teo"])
    y_log = np.log(tmp["close_bdr"])

    alpha_t, beta_t, sd_alpha_t, sd_beta_t, spread_kf_log = fit_kf_walkforward(
        x_log, y_log,
        q_alpha_grid=CFG["q_alpha_grid"],
        q_beta_grid=CFG["q_beta_grid"],
        r_grid=CFG["r_grid"],
        train=CFG["wfa_train"],
        test=CFG["wfa_test"],
        alpha0=CFG["init_alpha"], beta0=CFG["init_beta"],
        p0_alpha=CFG["init_p_alpha"], p0_beta=CFG["init_p_beta"],
        w_anchor=CFG["kf_w_anchor"], beta_anchor=CFG["kf_beta_anchor"], w_smooth=CFG["kf_w_smooth"]
    )

    tmp["alpha_t"]   = alpha_t
    tmp["beta_t"]    = beta_t
    tmp["sd_beta_t"] = sd_beta_t
    # (lag para trading)
    tmp["alpha_lag"] = tmp["alpha_t"].shift(1)
    tmp["beta_lag"]  = tmp["beta_t"].shift(1)

    # clip de β (lagged) para estabilidade
    beta_lo, beta_hi = CFG["kf_beta_clip"]
    beta_lag_clipped = tmp["beta_lag"].clip(lower=beta_lo, upper=beta_hi)

    # teórico dinâmico (preço): exp(α) * X^β  com α/β defasados
    tmp["bdr_teo_kf"] = np.exp(tmp["alpha_lag"]) * (tmp["bdr_teo"] ** beta_lag_clipped)

    # 5) Spreads e Z-scores
    spread_base = (tmp["close_bdr"] - tmp["bdr_teo"]).rename("spread_base")
    spread_kf   = (tmp["close_bdr"] - tmp["bdr_teo_kf"]).rename("spread_kf_price")
    tmp["z"]    = robust_zscore(spread_base, ewm_alpha=CFG["ewm_alpha"], winsor_pct=CFG["winsor_pct"])
    tmp["z_kf"] = robust_zscore(spread_kf,   ewm_alpha=CFG["ewm_alpha"], winsor_pct=CFG["winsor_pct"])

    # 6) Gate de cointegração (rolling) em LOGs + gate de estabilidade do KF
    tmp["coint_on"] = rolling_coint_gate(x_log, y_log, L=CFG["coint_L"], pval_th=CFG["coint_pval"])
    if CFG["use_kf_stability_gate"]:
        sd_ok = (tmp["sd_beta_t"].shift(1) <= CFG["kf_sd_beta_max"])
        beta_ok = (beta_lag_clipped == tmp["beta_lag"].shift(1).clip(beta_lo, beta_hi))  # True mesmo após clip
        tmp["gate_kf"] = (tmp["coint_on"] & sd_ok & beta_ok).fillna(False)
    else:
        tmp["gate_kf"] = tmp["coint_on"]

    # 7) Diagnósticos
    mae_base  = (tmp["close_bdr"] - tmp["bdr_teo"]).abs().mean()
    mae_kf    = (tmp["close_bdr"] - tmp["bdr_teo_kf"]).abs().mean()
    diag = diagnostics_quick(tmp)

    # 8) Salvar parquet do par
    out_path = out_pairs_dir / f"{bdr}_{adr}.parquet"
    tmp.reset_index().rename(columns={'index':'datetime'}).to_parquet(out_path, index=False)

    # 9) Plots (baseline)
    plot_prices(tmp, f"{bdr}_{adr}", plots_dir)
    plot_spread(tmp, f"{bdr}_{adr}", plots_dir)
    plot_zscore(tmp.rename(columns={"z":"z"}), f"{bdr}_{adr}", plots_dir)

    # Baseline
    bt_base, sum_base = backtest_next_bar(
        tmp,
        z_col="z",
        theo_col="bdr_teo",
        z_entry=CFG["z_enter"], z_exit=CFG["z_exit"], z_stop=CFG["z_stop"],
        slippage_bps=CFG["slippage_bps"], commission_bps=CFG["commission_bps"],
        borrow_bps_pa=CFG["borrow_bps_pa"], fx_basis_bps=CFG["fx_basis_bps"],
        use_risk_sizing=True, ewm_alpha=CFG["ewm_alpha"],
        target_vol=CFG["target_vol"], max_leverage=CFG["max_leverage"],
        gate=tmp["coint_on"],
        sd_beta=None, x_log=x_log,
        uncertainty_weight=CFG["uncertainty_weight"]
    )

    # Kalman (dinâmico)
    bt_kf, sum_kf = backtest_next_bar(
        tmp,
        z_col="z_kf",
        theo_col="bdr_teo_kf",
        z_entry=CFG["z_enter"], z_exit=CFG["z_exit"], z_stop=CFG["z_stop"],
        slippage_bps=CFG["slippage_bps"], commission_bps=CFG["commission_bps"],
        borrow_bps_pa=CFG["borrow_bps_pa"], fx_basis_bps=CFG["fx_basis_bps"],
        use_risk_sizing=True, ewm_alpha=CFG["ewm_alpha"],
        target_vol=CFG["target_vol"], max_leverage=CFG["max_leverage"],
        gate=tmp["gate_kf"],
        sd_beta=tmp["sd_beta_t"], x_log=x_log,
        uncertainty_weight=CFG["uncertainty_weight"]
    )

    # salvar curvas
    bt_base_path = out_pairs_dir / f"{bdr}_{adr}_bt_base.parquet"
    bt_kf_path   = out_pairs_dir / f"{bdr}_{adr}_bt_kf.parquet"
    bt_base.reset_index().rename(columns={'index':'datetime'}).to_parquet(bt_base_path, index=False)
    bt_kf.reset_index().rename(columns={'index':'datetime'}).to_parquet(bt_kf_path,   index=False)

    # resumo
    summary = {
        "bdr": bdr, "adr": adr,
        "overlap_obs": int(len(tmp)),
        "ratio_used": float(ratio_used),
        "ratio_method": ratio_method,
        "mae_teorico": float(mae_base),
        "mae_kf": float(mae_kf),
        "corr_log": float(diag["corr_log"]),
        "median_ratio_diag": float(diag["median_ratio"]),
        # backtests
        "sharpe_base": sum_base["sharpe_hourly_annualized"],
        "pnl_base": sum_base["pnl_total"],
        "mdd_base": sum_base["mdd"],
        "trades_base": sum_base["trades"],
        "sharpe_kf": sum_kf["sharpe_hourly_annualized"],
        "pnl_kf": sum_kf["pnl_total"],
        "mdd_kf": sum_kf["mdd"],
        "trades_kf": sum_kf["trades"],
    }

    log.info(f"[{bdr}_{adr}] BASE Sharpe={summary['sharpe_base']:.2f} ({summary['trades_base']} trades) | "
             f"KF Sharpe={summary['sharpe_kf']:.2f} ({summary['trades_kf']} trades) | overlap={summary['overlap_obs']}")
    return summary, out_path

# ============================ MAIN ============================

def main():
    OUT_DIR = Path(CFG["out_dir"])
    (OUT_DIR / "pairs").mkdir(parents=True, exist_ok=True)
    (OUT_DIR / "plots").mkdir(parents=True, exist_ok=True)

    # Carrega dados
    log.info("Lendo BDRs…")
    bdr_map = load_bdr_sheets(Path(CFG["path_bdr_xlsx"]))
    log.info(f"BDRs lidas: {len(bdr_map)}")

    log.info("Lendo ADRs…")
    us_map  = load_us_sheets(Path(CFG["path_us_xlsx"]))
    log.info(f"ADRs lidas: {len(us_map)}")

    log.info("Lendo FX (USDBRL)…")
    fx_df   = load_fx(Path(CFG["path_fx_xlsx"]))

    # Loop de pares
    rows_summary = []
    for bdr, adr in CFG["pairs"].items():
        if bdr not in bdr_map:
            log.warning(f"BDR '{bdr}' não encontrado; pulando.")
            continue
        if adr not in us_map:
            log.warning(f"ADR '{adr}' não encontrado; pulando.")
            continue

        res = run_pipeline_one_pair(
            bdr, adr, bdr_map, us_map, fx_df,
            out_pairs_dir=(OUT_DIR / "pairs"),
            plots_dir=(OUT_DIR / "plots")
        )
        if res is None:
            continue
        summary, _ = res
        rows_summary.append(summary)

    # Consolidado
    if rows_summary:
        df_sum = pd.DataFrame(rows_summary).sort_values("sharpe_kf", ascending=False)
        df_sum.to_csv(OUT_DIR / "summary_pairs.csv", index=False, float_format="%.8f")
        log.info("\n== Top 10 por Sharpe (KF) ==")
        cols = ["bdr","adr","overlap_obs","mae_teorico","mae_kf",
                "sharpe_base","pnl_base","trades_base",
                "sharpe_kf","pnl_kf","trades_kf","mdd_kf"]
        log.info("\n" + df_sum[cols].head(10).to_string(index=False))
    else:
        log.warning("Nenhum par válido processado.")

if __name__ == "__main__":
    main()


2025-10-22 09:45:31,378 INFO Lendo BDRs…
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/11/lm50x6ys7ng_15t73h0j41fm0000gn/T/ipykernel_8891/941683138.py", line 772, in <module>
    main()
  File "/var/folders/11/lm50x6ys7ng_15t73h0j41fm0000gn/T/ipykernel_8891/941683138.py", line 729, in main
    bdr_map = load_bdr_sheets(Path(CFG["path_bdr_xlsx"]))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/11/lm50x6ys7ng_15t73h0j41fm0000gn/T/ipykernel_8891/941683138.py", line 116, in load_bdr_sheets
    raw = pd.read_excel(path, sheet_name=None)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pandas/io/excel/_base.py", line 508, in read_excel
    data = io.parse(
           ^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pandas/io/excel/_base.py", line 1616, in parse
    re

In [ ]:
# %%
# -*- coding: utf-8 -*-
"""
ADR↔BDR — ETL + baseline teórico + Kalman (walk-forward) + backtest beta-neutral simplificado
(versão com ajustes de robustez do KF, sizing com incerteza de β e custos de execução)

Mudanças-chave (sem look-ahead):
- ρ calibrado só no primeiro TRAIN (evita forward-bias); α_t, β_t pelo Kalman com WFA + validação interna
- Penalização de var(Δβ) e |β-1| na escolha de (q_alpha, q_beta, r)
- Teórico dinâmico com β CLIPPED e LAGGED; gate_kf usa sd_beta e banda de β
- Sizing por risco: EWMA do spread + incerteza de β (lagged)
- Custos: borrow + FX basis + slippage & comissão via turnover
"""

from pathlib import Path
import sys, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint

# ============================ CONFIG & LOGGING ============================

CFG = {
    # dados
    "path_bdr_xlsx": "../data/Hist_BDRs.xlsx",
    "path_us_xlsx":  "../data/Hist_Origem_BDRs.xlsx",
    "path_fx_xlsx":  "../data/dolar.xlsx",
    "out_dir":       "../data/output",

    # pares (BDR -> ADR)
    "pairs": {
        "AAPL34": "AAPL",
        "MSFT34": "MSFT",
        "NVDC34": "NVDA",
        "AMZO34": "AMZN",
        "GOGL34": "GOOGL",
        "M1TA34": "META",
        "TSLA34": "TSLA",
        "TSMC34": "TSM",
        "AVGO34": "AVGO",
        "BABA34": "BABA",
        "JDCO34": "JD",
    },

    # Alinhamento temporal
    "shift_bdr_base_min": -30,   # -30min em todas as barras
    "shift_bdr_eod_min":  -60,   # -60min na última barra do dia
    "asof_tolerance": "31min",
    "min_overlap": 500,

    # Walk-Forward KF
    "wfa_train": 252,
    "wfa_test":  63,
    # grids mais conservadores (R maior ↓ reatividade)
    "q_alpha_grid": [1e-8, 1e-7, 1e-6, 1e-5],
    "q_beta_grid":  [1e-8, 1e-7, 1e-6, 1e-5],
    "r_grid":       [1e-4, 1e-3, 1e-2, 1e-1],
    "init_alpha": 0.0,
    "init_beta":  1.0,
    "init_p_alpha": 1.0,
    "init_p_beta":  1.0,

    # Regularização/penalização na seleção do KF
    "kf_beta_anchor": 1.0,
    "kf_w_anchor": 0.05,     # penaliza |β-1|
    "kf_w_smooth": 0.10,     # penaliza var(Δβ)

    # Clip/estabilidade do β e gate
    "kf_beta_clip": (0.3, 2.0),
    "kf_sd_beta_max": 0.25,  # em log-space
    "use_kf_stability_gate": True,

    # Z-Score robusto
    "winsor_pct": 0.01,
    "ewm_alpha": 0.06,  # ~half-life 16 barras

    # Gate de cointegração
    "coint_L": 252,
    "coint_pval": 0.05,

    # Sinais & sizing
    "z_enter": 2.0,
    "z_exit": 0.75,
    "z_stop": 3.0,
    "target_vol": 0.08,
    "max_leverage": 2.0,
    "uncertainty_weight": 1.0,  # peso da incerteza de β no sizing

    # Custos
    "slippage_bps": 3.0,
    "commission_bps": 3.0,
    "borrow_bps_pa": 400.0,  # aluguel short BDR
    "fx_basis_bps": 20.0,    # custo hedge FX aprox

    # Diversos
    "plot_top_n": None,
    
    # === HMM (gate de regimes) ===
    "use_hmm_gate": True,
    "hmm_obs": "dspread_kf",   # opções: "dspread_kf", "spread_kf", "z_kf", "spread", "z"
    "hmm_n_states": 2,
    "hmm_max_iter": 100,
    "hmm_tol": 1e-4,
    "hmm_p_stay_init": 0.95,   # matriz de transição inicial quase diagonal
    "hmm_p_threshold": 0.6,    # P(MR) mínima para liberar trade

}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
log = logging.getLogger("pairs")

# ============================ I/O DE DADOS ============================

def _ptbr_to_float(series: pd.Series) -> pd.Series:
    try:
        return series.astype(float)
    except Exception:
        return pd.to_numeric(series, errors='coerce')

def load_bdr_sheets(path: Path) -> dict[str, pd.DataFrame]:
    raw = pd.read_excel(path, sheet_name=None)
    out = {}
    for sheet, df in raw.items():
        if 'Data' not in df.columns or 'Fechamento' not in df.columns:
            continue
        df = df.rename(columns={'Data':'datetime','Fechamento':'close_bdr','Volume Financeiro':'volume_bdr'})
        df['datetime']  = pd.to_datetime(df['datetime'], dayfirst=True)
        df['close_bdr'] = _ptbr_to_float(df['close_bdr'])
        if 'volume_bdr' in df.columns:
            df['volume_bdr'] = _ptbr_to_float(df['volume_bdr'])
        else:
            df['volume_bdr'] = np.nan
        df = (df[['datetime','close_bdr','volume_bdr']]
                .dropna(subset=['datetime','close_bdr'])
                .sort_values('datetime')
                .set_index('datetime'))
        out[sheet] = df
    return out

def load_us_sheets(path: Path) -> dict[str, pd.DataFrame]:
    raw = pd.read_excel(path, sheet_name=None)
    out = {}
    for sheet, df in raw.items():
        if 'Date' not in df.columns or 'Last Price' not in df.columns:
            continue
        df = df.rename(columns={'Date':'datetime','Last Price':'close_us','Volume':'volume_us'})
        df['datetime']  = pd.to_datetime(df['datetime'], dayfirst=True)
        df['close_us']  = _ptbr_to_float(df['close_us'])
        if 'volume_us' in df.columns:
            df['volume_us'] = _ptbr_to_float(df['volume_us'])
        else:
            df['volume_us'] = np.nan
        df = (df[['datetime','close_us','volume_us']]
                .dropna(subset=['datetime','close_us'])
                .sort_values('datetime')
                .set_index('datetime'))
        out[sheet] = df
    return out

def load_fx(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    if 'Date' not in df.columns or 'Mid Price' not in df.columns:
        raise ValueError("dolar.xlsx precisa conter colunas 'Date' e 'Mid Price'.")
    df = df.rename(columns={'Date':'datetime','Mid Price':'usdxbrl','Volume':'volume_fx'})
    df['datetime'] = pd.to_datetime(df['datetime'], dayfirst=True)
    df['usdxbrl']  = _ptbr_to_float(df['usdxbrl'])
    if 'volume_fx' in df.columns:
        df['volume_fx'] = _ptbr_to_float(df['volume_fx'])
    else:
        df['volume_fx'] = np.nan
    df = (df[['datetime','usdxbrl','volume_fx']]
            .dropna(subset=['datetime','usdxbrl'])
            .sort_values('datetime')
            .set_index('datetime'))
    return df

# ============================ ALINHAMENTO & TEÓRICO ============================

def shift_bdr_index_with_eod_rule(df_bdr: pd.DataFrame,
                                  base_min: int = -30,
                                  eod_min:  int = -60) -> pd.DataFrame:
    out = df_bdr.copy()
    idx = out.index
    idx_s = pd.Series(idx, index=idx)
    last_per_day = idx_s.groupby(idx_s.index.normalize()).transform('max')
    is_last = (idx_s == last_per_day)
    base_delta = pd.to_timedelta(base_min, unit='m')
    eod_delta  = pd.to_timedelta(eod_min,  unit='m')
    new_idx = pd.DatetimeIndex(idx + base_delta)
    new_idx = pd.DatetimeIndex(np.where(is_last.values,
                                        (idx + eod_delta).values,
                                        new_idx.values))
    out.index = new_idx
    return out.sort_index()

def nearest_join(left: pd.DataFrame, right: pd.DataFrame, tolerance: str, direction: str = "backward") -> pd.DataFrame:
    l = left.sort_index().reset_index().rename(columns={'index':'datetime'})
    r = right.sort_index().reset_index().rename(columns={'index':'datetime'})
    m = pd.merge_asof(l, r, on='datetime',
                      direction=direction,
                      tolerance=pd.to_timedelta(tolerance))
    m = m.set_index('datetime')
    return m

def theoretical_bdr_brl(px_adr_usd: pd.Series,
                        fx_brlusd: pd.Series,
                        rho: float) -> pd.Series:
    return (px_adr_usd * fx_brlusd * float(rho)).rename("theo_bdr_brl")

# ============================ RATIO (ρ) ============================

def calibrate_ratio(bdr_close: pd.Series, adr_usd: pd.Series, usdxbrl: pd.Series, method="ols"):
    denom = adr_usd * usdxbrl
    df = pd.DataFrame({'bdr': bdr_close, 'den': denom}).dropna()
    if df.empty: return np.nan, np.nan, 0
    ratio_series = df['bdr'] / df['den'].replace(0, np.nan)
    ratio_series = ratio_series.replace([np.inf, -np.inf], np.nan).dropna()
    if ratio_series.empty: return np.nan, np.nan, int(len(df))
    if method == "median":
        ratio = float(np.median(ratio_series.values))
    else:
        num = (df['den'] * df['bdr']).sum()
        den = (df['den'] ** 2).sum()
        ratio = float(num / den) if den != 0 else np.nan
    return ratio, float(ratio_series.std(ddof=1)), int(len(ratio_series))

def choose_ratio_with_fallback(df_pair: pd.DataFrame, ratio_ols: float):
    denom = df_pair['close_us'] * df_pair['usdxbrl']
    ratio_series = (df_pair['close_bdr'] / denom).replace([np.inf,-np.inf], np.nan).dropna()
    if ratio_series.empty:
        return ratio_ols, np.nan, 0, "ols"
    ratio_med = float(np.median(ratio_series))
    ratio_sd  = float(ratio_series.std(ddof=1)) if len(ratio_series) > 1 else 0.0
    if (ratio_med > 0) and (ratio_ols <= 0 or ratio_ols < ratio_med/100.0 or ratio_ols > ratio_med*100.0):
        return ratio_med, ratio_sd, len(ratio_series), "median_fallback"
    return ratio_ols, ratio_sd, len(ratio_series), "ols"

# ============================ ESTATÍSTICA ROBUSTA ============================

def robust_zscore(spread: pd.Series, ewm_alpha=0.06, winsor_pct=0.01) -> pd.Series:
    s = spread.copy()
    lo, hi = s.quantile([winsor_pct, 1-winsor_pct])
    s = s.clip(lo, hi)
    mu = s.ewm(alpha=ewm_alpha, adjust=False).mean().shift(1)
    dev = (s - mu).abs().ewm(alpha=ewm_alpha, adjust=False).mean().shift(1)
    eps = 1e-12
    return ((s - mu) / (dev + eps)).rename("z")

# ============================ COINTEGRAÇÃO (gate) ============================

def rolling_coint_gate(x: pd.Series, y: pd.Series, L=252, pval_th=0.05) -> pd.Series:
    gate = pd.Series(False, index=y.index)
    if len(y) < L: return gate
    for i in range(L, len(y)):
        _x = x.iloc[i-L:i].values
        _y = y.iloc[i-L:i].values
        try:
            _, pval, _ = coint(_y, _x)
            gate.iloc[i] = (pval < pval_th)
        except Exception:
            gate.iloc[i] = False
    return gate.rename("coint_on")

# ============================ KALMAN (manual, 2D) ============================

def kalman_alpha_beta(y: pd.Series, x: pd.Series,
                      q_alpha=1e-5, q_beta=1e-5, r=1e-4,
                      alpha0=0.0, beta0=1.0, p0_alpha=1.0, p0_beta=1.0):
    """
    Estado 2D: y_t = α_t + β_t x_t + e_t   (tudo em LOGs)
    Retorna DataFrame com alpha_t, beta_t, sd_alpha_t, sd_beta_t, spread_kf (inovação).
    """
    y = pd.Series(y).astype(float)
    x = pd.Series(x).astype(float)
    idx = y.index.intersection(x.index)
    y = y.loc[idx]; x = x.loc[idx]

    theta = np.array([alpha0, beta0], dtype=float)
    P = np.diag([p0_alpha, p0_beta])
    Q = np.diag([q_alpha, q_beta])
    R = np.array([[r]])

    alphas, betas, sd_a, sd_b, resid = [], [], [], [], []

    for t in range(len(idx)):
        H = np.array([[1.0, float(x.iloc[t])]])
        # Predição
        theta_pred = theta
        P_pred = P + Q
        # Inovação
        y_hat = float(H @ theta_pred)
        nu = float(y.iloc[t] - y_hat)
        S = float(H @ P_pred @ H.T + R)
        K = (P_pred @ H.T) / S
        # Atualização
        theta = theta_pred + (K.flatten() * nu)
        P = (np.eye(2) - K @ H) @ P_pred

        alphas.append(theta[0]); betas.append(theta[1])
        sd_a.append(np.sqrt(max(P[0,0], 0.0))); sd_b.append(np.sqrt(max(P[1,1], 0.0)))
        resid.append(nu)

    out = pd.DataFrame({
        "alpha_t": alphas,
        "beta_t":  betas,
        "sd_alpha_t": sd_a,
        "sd_beta_t":  sd_b,
        "spread_kf":  resid,
    }, index=idx)
    return out

def fit_kf_walkforward(x_log: pd.Series, y_log: pd.Series,
                       q_alpha_grid, q_beta_grid, r_grid,
                       train=252, test=63,
                       alpha0=0.0, beta0=1.0, p0_alpha=1.0, p0_beta=1.0,
                       w_anchor=0.05, beta_anchor=1.0, w_smooth=0.10):
    """
    WFA com validação interna:
      - Para cada janela TRAIN, escolhe (q_alpha, q_beta, r) que minimiza:
        OBJ = RMSE(residual) na *cauda* do TRAIN (últimos 20%)
              + w_anchor * mean((β-β_anchor)^2)
              + w_smooth * var(Δβ)
      - Reaplica KF em TRAIN+TEST com hiperparâmetros escolhidos e coleta α,β,sd's no TEST.
    """
    idx = y_log.index
    a_full = pd.Series(index=idx, dtype=float)
    b_full = pd.Series(index=idx, dtype=float)
    sa_full = pd.Series(index=idx, dtype=float)
    sb_full = pd.Series(index=idx, dtype=float)
    s_full = pd.Series(index=idx, dtype=float)

    i = train
    while i + test <= len(idx):
        tr = slice(idx[i-train], idx[i-1])
        te = slice(idx[i], idx[i+test-1])

        x_tr, y_tr = x_log.loc[tr], y_log.loc[tr]
        x_te, y_te = x_log.loc[te], y_log.loc[te]

        # parte de validação dentro do TRAIN (últimos 20%)
        cut = int(round(0.8 * len(x_tr)))
        x_val, y_val = x_tr.iloc[cut:], y_tr.iloc[cut:]

        best = None
        for qa in q_alpha_grid:
            for qb in q_beta_grid:
                for rr in r_grid:
                    kf_tr = kalman_alpha_beta(
                        y_tr, x_tr, q_alpha=qa, q_beta=qb, r=rr,
                        alpha0=alpha0, beta0=beta0, p0_alpha=p0_alpha, p0_beta=p0_beta
                    )
                    # métrica na cauda do TRAIN (proxy 'quase-OOS')
                    kf_tail = kf_tr.iloc[cut:].copy()
                    rmse_tail = float(np.sqrt(np.nanmean((kf_tail["spread_kf"].values)**2)))
                    # regularizações de estabilidade de β
                    beta_series = kf_tr["beta_t"].values
                    dbeta = np.diff(beta_series)
                    smooth_pen = np.nanvar(dbeta) if len(dbeta)>1 else 0.0
                    anchor_pen = np.nanmean((beta_series - beta_anchor)**2)
                    obj = rmse_tail + w_anchor*anchor_pen + w_smooth*smooth_pen

                    if (best is None) or (obj < best["obj"]):
                        best = {"qa": qa, "qb": qb, "rr": rr, "obj": obj}

        # Refit em TRAIN+TEST com os hiperparâmetros escolhidos
        x_block = pd.concat([x_tr, x_te], axis=0)
        y_block = pd.concat([y_tr, y_te], axis=0)
        kf_block = kalman_alpha_beta(
            y_block, x_block,
            q_alpha=best["qa"], q_beta=best["qb"], r=best["rr"],
            alpha0=alpha0, beta0=beta0, p0_alpha=p0_alpha, p0_beta=p0_beta
        )
        kk = kf_block.loc[te]
        a_full.loc[te]  = kk["alpha_t"]
        b_full.loc[te]  = kk["beta_t"]
        sa_full.loc[te] = kk["sd_alpha_t"]
        sb_full.loc[te] = kk["sd_beta_t"]
        s_full.loc[te]  = kk["spread_kf"]

        i += test

    return (a_full.rename("alpha_t"),
            b_full.rename("beta_t"),
            sa_full.rename("sd_alpha_t"),
            sb_full.rename("sd_beta_t"),
            s_full.rename("spread_kf"))


# ============================ KMM  ============================
def hmm_gaussian_em(y: np.ndarray,
                    n_states: int = 2,
                    max_iter: int = 100,
                    tol: float = 1e-4,
                    p_stay_init: float = 0.95):
    """
    EM (Baum–Welch) para HMM Gaussiano 1D.
    Retorna dict com: pi (K,), A (K,K), mu (K,), var (K,), loglik.
    """
    y = np.asarray(y, float)
    T = len(y); K = int(n_states)
    if T < K+2:  # proteção
        raise ValueError("Série curta para HMM.")

    # Inicializações simples e robustas
    pi = np.full(K, 1.0/K)
    A  = np.full((K, K), (1 - p_stay_init) / (K - 1))
    np.fill_diagonal(A, p_stay_init)

    # médias em quantis, variâncias iguais
    qs = np.linspace(0.0, 1.0, K+2)[1:-1]
    mu = np.quantile(y, qs) if K > 1 else np.array([np.median(y)])
    var = np.full(K, np.var(y) + 1e-6)

    prev_ll = -np.inf
    for _ in range(max_iter):
        # ---- E-step: forward-backward em log ----
        # log-emissões para todos os estados
        logB = np.vstack([_norm_logpdf(y, mu[k], var[k]) for k in range(K)]).T  # (T,K)

        # forward (log-alpha)
        log_alpha = np.empty((T, K))
        log_alpha[0] = np.log(pi) + logB[0]
        for t in range(1, T):
            log_alpha[t] = logB[t] + _logsumexp(log_alpha[t-1][:,None] + np.log(A), axis=0)

        # log verossimilhança
        ll = _logsumexp(log_alpha[-1], axis=0)

        # backward (log-beta)
        log_beta = np.zeros((T, K))
        for t in range(T-2, -1, -1):
            log_beta[t] = _logsumexp(np.log(A) + (logB[t+1] + log_beta[t+1])[None,:], axis=1)

        # gammas/xis (em prob.)
        log_gamma = log_alpha + log_beta - ll
        gamma = np.exp(log_gamma)                           # (T,K)
        xi = np.zeros((T-1, K, K))
        for t in range(T-1):
            z = (log_alpha[t][:,None] + np.log(A) + (logB[t+1] + log_beta[t+1])[None,:])
            z -= _logsumexp(z, axis=None)  # norma total
            xi[t] = np.exp(z)

        # ---- M-step ----
        pi = np.clip(gamma[0], 1e-12, None); pi /= pi.sum()
        A = np.clip(xi.sum(axis=0), 1e-12, None)
        A /= A.sum(axis=1, keepdims=True)

        w = np.clip(gamma.sum(axis=0), 1e-12, None)        # (K,)
        mu = (gamma * y[:,None]).sum(axis=0) / w
        var = (gamma * ((y[:,None]-mu)**2)).sum(axis=0) / w
        var = np.maximum(var, 1e-8)

        # critério de parada
        if ll - prev_ll < tol:
            break
        prev_ll = ll

    return {"pi": pi, "A": A, "mu": mu, "var": var, "loglik": float(ll)}

def hmm_filter_proba(y: np.ndarray, params: dict) -> np.ndarray:
    """
    Probabilidade filtrada p(s_t=k | y_1:t) (sem look-ahead).
    Retorna (T,K). Usa somente forward normalizado.
    """
    y = np.asarray(y, float)
    pi, A, mu, var = params["pi"], params["A"], params["mu"], params["var"]
    T = len(y); K = len(pi)

    logB = np.vstack([_norm_logpdf(y, mu[k], var[k]) for k in range(K)]).T
    log_alpha = np.empty((T, K))
    log_alpha[0] = np.log(pi) + logB[0]
    for t in range(1, T):
        log_alpha[t] = logB[t] + _logsumexp(log_alpha[t-1][:,None] + np.log(A), axis=0)

    # normaliza em cada t -> probas filtradas
    log_alpha -= _logsumexp(log_alpha, axis=1)[:,None]
    return np.exp(log_alpha)  # (T,K)

def fit_hmm_walkforward(obs: pd.Series,
                        train: int = 252,
                        test: int = 63,
                        n_states: int = 2,
                        max_iter: int = 100,
                        tol: float = 1e-4,
                        p_stay_init: float = 0.95):
    """
    Treina HMM em janelas TRAIN e devolve p(MR) filtrada (apenas no TEST).
    Estado 'MR' = argmin_k [ var_k + |mu_k| ]  (baixa variância e pouco viés).
    """
    obs = pd.Series(obs).astype(float).dropna()
    idx = obs.index
    p_mr_full = pd.Series(index=idx, dtype=float)

    i = train
    while i + test <= len(idx):
        tr = slice(idx[i-train], idx[i-1])
        te = slice(idx[i], idx[i+test-1])
        y_tr = obs.loc[tr].values
        y_block = obs.loc[tr].append(obs.loc[te]).values  # train+test

        try:
            pars = hmm_gaussian_em(y_tr, n_states=n_states,
                                   max_iter=max_iter, tol=tol,
                                   p_stay_init=p_stay_init)
        except Exception:
            i += test
            continue

        # estado MR pelo score var+|mu|
        score = pars["var"] + np.abs(pars["mu"])
        k_mr = int(np.argmin(score))

        # probabilidades filtradas no bloco todo e coleta do TEST
        P = hmm_filter_proba(y_block, pars)  # (T_block,K)
        Ttr = len(y_tr)
        p_mr = P[Ttr:, k_mr]                 # somente TEST
        p_mr_full.loc[te] = p_mr

        i += test

    return p_mr_full.rename("p_mr")


def _logsumexp(a: np.ndarray, axis=None):
    a_max = np.max(a, axis=axis, keepdims=True)
    out = a_max + np.log(np.sum(np.exp(a - a_max), axis=axis, keepdims=True))
    return np.squeeze(out, axis=axis)

def _norm_logpdf(y: np.ndarray, mu: np.ndarray, var: np.ndarray, eps: float = 1e-12):
    var = np.maximum(var, eps)
    return -0.5 * (np.log(2*np.pi) + np.log(var) + ((y - mu)**2)/var)

# ============================ MÉTRICAS/DIAGNÓSTICO ============================

def estimate_bars_per_day_from_index(index: pd.DatetimeIndex) -> int:
    if len(index) == 0: return 1
    counts = pd.Series(index.normalize()).value_counts()
    return int(np.median(counts.values))

def diagnostics_quick(df_pair: pd.DataFrame):
    x = np.log(df_pair['close_bdr'])
    y = np.log(df_pair['close_us'] * df_pair['usdxbrl'])
    valid = x.notna() & y.notna()
    corr = x[valid].corr(y[valid]) if valid.any() else np.nan
    ratio_inst = (df_pair['close_bdr'] / (df_pair['close_us'] * df_pair['usdxbrl'])).replace([np.inf,-np.inf], np.nan)
    med_ratio = float(ratio_inst.median())
    return {"corr_log": float(corr), "median_ratio": med_ratio, "n": int(valid.sum())}

# ============================ BACKTEST (next-bar, custos) ============================

def backtest_next_bar(df: pd.DataFrame,
                      z_col="z",
                      theo_col="bdr_teo",
                      z_entry=2.0, z_exit=0.75, z_stop=3.0,
                      slippage_bps=0.0, commission_bps=0.0,
                      borrow_bps_pa=400.0, fx_basis_bps=20.0,
                      use_risk_sizing=True, ewm_alpha=0.06,
                      target_vol=0.08, max_leverage=2.0,
                      gate: pd.Series | None = None,
                      sd_beta: pd.Series | None = None,
                      x_log: pd.Series | None = None,
                      uncertainty_weight: float = 1.0):
    """
    Backtest do spread S = BDR - Teórico (preço), execução NEXT-BAR.
    Sizing por risco usa EWMA(std(S)) e, opcionalmente, adiciona incerteza de β (lagged):

        var_eff_t = var_EWMA(S)_t  +  (theo_{t-1} * ln(X_t))^2 * Var(β_{t-1})

    onde X_t = teórico estático (ADR×FX×ρ) em preço; Var(β) ≈ sd_beta^2.

    Custos:
      - slippage + comissão por turnover |Δpos|*(pxB+pxT) * (bps/1e4)
      - aluguel short BDR (sobre notional da perna BDR)
      - FX basis (sobre notional da perna sintética)
    """
    df = df.copy().sort_index()

    S   = (df["close_bdr"] - df[theo_col]).rename("S")
    z   = df[z_col].copy()
    pxB = df["close_bdr"]
    pxT = df[theo_col]

    # Gate
    if gate is not None:
        gate = gate.reindex(df.index).fillna(False)
    else:
        gate = pd.Series(True, index=df.index)

    # var efetiva (EWMA do spread + incerteza beta)
    if use_risk_sizing:
        base_var = (S - S.ewm(alpha=ewm_alpha, adjust=False).mean()).pow(2).ewm(alpha=ewm_alpha, adjust=False).mean()
        extra = 0.0
        if (sd_beta is not None) and (x_log is not None):
            sd_b_lag = sd_beta.reindex(df.index).shift(1).fillna(method="ffill").fillna(0.0)
            lnX = x_log.reindex(df.index).fillna(method="ffill").fillna(0.0)  # log do teórico estático
            theo_lag = pxT.shift(1).fillna(method="ffill").fillna(pxT.iloc[0])
            extra = (theo_lag * lnX).pow(2) * (sd_b_lag.pow(2)) * float(uncertainty_weight)
        var_eff = (base_var + extra).replace(0, np.nan)
        vol_eff = np.sqrt(var_eff).shift(1)
        w = (target_vol / vol_eff).clip(upper=max_leverage).fillna(0.0)
    else:
        w = pd.Series(target_vol, index=df.index).clip(upper=max_leverage)

    # estado discreto
    state = pd.Series(0, index=df.index, dtype=int)
    pos = 0
    for t, zz in z.items():
        if not gate.loc[t]:
            pos = 0
        else:
            if pos == 0:
                if zz >  z_entry: pos = -1
                if zz < -z_entry: pos = +1
            else:
                if abs(zz) > z_stop:
                    pos = 0
                elif abs(zz) < z_exit:
                    pos = 0
        state.loc[t] = pos

    # posição efetiva (contínua)
    pos_w = (state * w).rename("pos")

    # Execução next-bar
    pos_exec = pos_w.shift(1).fillna(0.0)
    dS = S.diff().fillna(0.0)
    pnl_gross = (pos_exec * dS).rename("pnl_gross")

    # Turnover e custos de slippage/comissão (por mudança de posição)
    dpos = pos_exec.diff().abs().fillna(pos_exec.abs())
    bps_trading = (slippage_bps + commission_bps) / 1e4
    trade_cost = (dpos * (pxB.shift(1).fillna(pxB) + pxT.shift(1).fillna(pxT)) * bps_trading).rename("trade_cost")

    # Custos recorrentes (diários aprox)
    days = (df.index.to_series().diff().dt.days.fillna(0)).clip(lower=0)
    borrow_daily = (borrow_bps_pa/1e4) / 252.0
    fx_basis_daily = (fx_basis_bps/1e4) / 252.0

    borrow_cost = (pos_exec.clip(upper=0).abs() * pxB.shift(1) * borrow_daily * (days.replace(0,1))).rename("borrow_cost")
    fx_cost     = (pos_exec.abs() * pxT.shift(1) * fx_basis_daily * (days.replace(0,1))).rename("fx_cost")

    pnl_net = (pnl_gross - trade_cost - borrow_cost - fx_cost).rename("pnl_net")
    equity  = pnl_net.cumsum().rename("equity")

    # métricas
    bars_per_day = estimate_bars_per_day_from_index(df.index)
    ann_factor = np.sqrt(max(bars_per_day,1) * 252.0)
    ret = pnl_net
    std = float(ret.std(ddof=1))
    sharpe = (float(ret.mean()) / (std + 1e-12)) * ann_factor if std > 0 else np.nan
    roll_max = equity.cummax()
    mdd = float((equity - roll_max).min()) if len(equity) else np.nan

    # estatística de trades
    turns = (state.shift(1).fillna(0) != state).astype(int)
    trade_idx = turns[turns==1].index
    trades = int((state.abs().diff().fillna(state.abs()) > 0).sum())
    # winrate/expectancy aproximados por janelas entre flips
    wins = 0; losses = 0; pnl_trades = []
    if len(trade_idx) > 1:
        starts = trade_idx
        stops = list(starts[1:]) + [df.index[-1]]
        for s, e in zip(starts, stops):
            seg = equity.loc[s:e]
            if len(seg) >= 2:
                dp = float(seg.iloc[-1] - seg.iloc[0])
                pnl_trades.append(dp)
                if dp > 0: wins += 1
                elif dp < 0: losses += 1
    winrate = (wins / max(wins+losses,1)) if (wins+losses)>0 else np.nan
    expectancy = float(np.nanmedian(pnl_trades)) if len(pnl_trades)>0 else np.nan
    # holding mediano em barras
    hold_lengths = []
    if len(trade_idx) > 1:
        for s, e in zip(trade_idx, list(trade_idx[1:]) + [df.index[-1]]):
            hold_lengths.append(max(1, (df.index.get_loc(e) - df.index.get_loc(s))))
    med_hold = int(np.median(hold_lengths)) if hold_lengths else 0

    out = pd.concat([S, z, pos_w, pnl_gross, trade_cost, borrow_cost, fx_cost, pnl_net, equity], axis=1)
    summary = {
        "bars": len(df),
        "trades": trades,
        "winrate": winrate,
        "expectancy_per_trade": expectancy,
        "median_hold_bars": med_hold,
        "sharpe_hourly_annualized": sharpe,
        "mdd": mdd,
        "pnl_total": float(equity.iloc[-1]) if len(equity) else 0.0,
    }
    return out, summary

# ============================ PLOTS (opcional) ============================

def plot_prices(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    df[['close_bdr','bdr_teo']].dropna().plot(ax=plt.gca())
    plt.title(f"{name} — BDR vs Teórico")
    plt.xlabel("Tempo"); plt.ylabel("Preço (BRL)")
    fig.tight_layout(); fig.savefig(outdir / f"{name}_prices.png"); plt.close(fig)

def plot_spread(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    (df['close_bdr'] - df['bdr_teo']).dropna().plot(ax=plt.gca())
    plt.title(f"{name} — Spread (BDR − Teórico)")
    plt.xlabel("Tempo"); plt.ylabel("Spread (BRL)")
    fig.tight_layout(); fig.savefig(outdir / f"{name}_spread.png"); plt.close(fig)

def plot_zscore(df: pd.DataFrame, name: str, outdir: Path):
    fig = plt.figure()
    df['z'].dropna().plot(ax=plt.gca())
    plt.title(f"{name} — Z-Score do Spread (robusto, shift(1))")
    plt.xlabel("Tempo"); plt.ylabel("Z")
    ax = plt.gca(); ax.axhline(0); ax.axhline(2); ax.axhline(-2)
    fig.tight_layout(); fig.savefig(outdir / f"{name}_zscore.png"); plt.close(fig)

# ============================ PIPELINE ============================

def run_pipeline_one_pair(bdr: str, adr: str,
                          bdr_map: dict, us_map: dict, fx_df: pd.DataFrame,
                          out_pairs_dir: Path, plots_dir: Path):
    # 1) Shift do BDR
    df_bdr = shift_bdr_index_with_eod_rule(
        bdr_map[bdr],
        base_min=CFG["shift_bdr_base_min"],
        eod_min=CFG["shift_bdr_eod_min"],
    )

    # 2) Nearest join BDR↔ADR↔FX (backward)
    tmp = nearest_join(
        df_bdr[['close_bdr','volume_bdr']],
        us_map[adr][['close_us','volume_us']],
        CFG["asof_tolerance"]
    )
    tmp = nearest_join(tmp, fx_df[['usdxbrl','volume_fx']], CFG["asof_tolerance"])
    tmp = tmp.dropna(subset=['close_bdr','close_us','usdxbrl']).sort_index()

    if len(tmp) < CFG["min_overlap"]:
        log.info(f"[{bdr}↔{adr}] overlap insuficiente: {len(tmp)}")
        return None

    # 3) ρ (ratio) usando apenas o primeiro TRAIN (evita look-ahead)
    n_train = min(CFG["wfa_train"], len(tmp)//3 or 1)
    tmp_head = tmp.iloc[:n_train]
    ratio_ols, ratio_sd_raw, n_ratio_raw = calibrate_ratio(
        tmp_head['close_bdr'], tmp_head['close_us'], tmp_head['usdxbrl'], method="ols"
    )
    ratio_used, ratio_sd, n_ratio, ratio_method = choose_ratio_with_fallback(tmp_head, ratio_ols)
    tmp["bdr_teo"] = theoretical_bdr_brl(tmp["close_us"], tmp["usdxbrl"], ratio_used)

    # 4) KF Walk-Forward (α_t, β_t) sobre LOGs (com validação interna e penalizações)
    x_log = np.log(tmp["bdr_teo"])
    y_log = np.log(tmp["close_bdr"])

    alpha_t, beta_t, sd_alpha_t, sd_beta_t, spread_kf_log = fit_kf_walkforward(
        x_log, y_log,
        q_alpha_grid=CFG["q_alpha_grid"],
        q_beta_grid=CFG["q_beta_grid"],
        r_grid=CFG["r_grid"],
        train=CFG["wfa_train"],
        test=CFG["wfa_test"],
        alpha0=CFG["init_alpha"], beta0=CFG["init_beta"],
        p0_alpha=CFG["init_p_alpha"], p0_beta=CFG["init_p_beta"],
        w_anchor=CFG["kf_w_anchor"], beta_anchor=CFG["kf_beta_anchor"], w_smooth=CFG["kf_w_smooth"]
    )

    tmp["alpha_t"]   = alpha_t
    tmp["beta_t"]    = beta_t
    tmp["sd_beta_t"] = sd_beta_t
    # (lag para trading)
    tmp["alpha_lag"] = tmp["alpha_t"].shift(1)
    tmp["beta_lag"]  = tmp["beta_t"].shift(1)

    # clip de β (lagged) para estabilidade
    beta_lo, beta_hi = CFG["kf_beta_clip"]
    beta_lag_clipped = tmp["beta_lag"].clip(lower=beta_lo, upper=beta_hi)

    # teórico dinâmico (preço): exp(α) * X^β  com α/β defasados
    tmp["bdr_teo_kf"] = np.exp(tmp["alpha_lag"]) * (tmp["bdr_teo"] ** beta_lag_clipped)

    # 5) Spreads e Z-scores
    spread_base = (tmp["close_bdr"] - tmp["bdr_teo"]).rename("spread_base")
    spread_kf   = (tmp["close_bdr"] - tmp["bdr_teo_kf"]).rename("spread_kf_price")
    tmp["z"]    = robust_zscore(spread_base, ewm_alpha=CFG["ewm_alpha"], winsor_pct=CFG["winsor_pct"])
    tmp["z_kf"] = robust_zscore(spread_kf,   ewm_alpha=CFG["ewm_alpha"], winsor_pct=CFG["winsor_pct"])

    # 6) Gate de cointegração (rolling) em LOGs + gate de estabilidade do KF
    tmp["coint_on"] = rolling_coint_gate(x_log, y_log, L=CFG["coint_L"], pval_th=CFG["coint_pval"])
    if CFG["use_kf_stability_gate"]:
        sd_ok = (tmp["sd_beta_t"].shift(1) <= CFG["kf_sd_beta_max"])
        beta_ok = (beta_lag_clipped == tmp["beta_lag"].shift(1).clip(beta_lo, beta_hi))  # True mesmo após clip
        tmp["gate_kf"] = (tmp["coint_on"] & sd_ok & beta_ok).fillna(False)
    else:
        tmp["gate_kf"] = tmp["coint_on"]

    # === HMM regime gate (opcional) ===
    if CFG.get("use_hmm_gate", False):
        # escolhe observável para o HMM
        obs_map = {
            "dspread_kf": spread_kf.diff().fillna(0.0),
            "spread_kf":  spread_kf,
            "z_kf":       tmp["z_kf"],
            "spread":     spread_base,
            "z":          tmp["z"],
        }
        obs_hmm = obs_map.get(CFG["hmm_obs"], spread_kf.diff().fillna(0.0)).dropna()

        p_mr = fit_hmm_walkforward(
            obs_hmm,
            train=CFG["wfa_train"],
            test=CFG["wfa_test"],
            n_states=CFG["hmm_n_states"],
            max_iter=CFG["hmm_max_iter"],
            tol=CFG["hmm_tol"],
            p_stay_init=CFG["hmm_p_stay_init"]
        )
        # alinha e defasa (evita look-ahead)
        tmp["p_mr"] = p_mr.reindex(tmp.index)
        tmp["gate_hmm"] = (tmp["p_mr"].shift(1) >= CFG["hmm_p_threshold"]).fillna(False)

        # combina com gate do KF
        tmp["gate_kf_hmm"] = (tmp["gate_kf"] & tmp["gate_hmm"]).fillna(False)

    # 7) Diagnósticos
    mae_base  = (tmp["close_bdr"] - tmp["bdr_teo"]).abs().mean()
    mae_kf    = (tmp["close_bdr"] - tmp["bdr_teo_kf"]).abs().mean()
    diag = diagnostics_quick(tmp)

    # 8) Salvar parquet do par
    out_path = out_pairs_dir / f"{bdr}_{adr}.parquet"
    tmp.reset_index().rename(columns={'index':'datetime'}).to_parquet(out_path, index=False)

    # 9) Plots (baseline)
    plot_prices(tmp, f"{bdr}_{adr}", plots_dir)
    plot_spread(tmp, f"{bdr}_{adr}", plots_dir)
    plot_zscore(tmp.rename(columns={"z":"z"}), f"{bdr}_{adr}", plots_dir)

    # Baseline
    bt_base, sum_base = backtest_next_bar(
        tmp,
        z_col="z",
        theo_col="bdr_teo",
        z_entry=CFG["z_enter"], z_exit=CFG["z_exit"], z_stop=CFG["z_stop"],
        slippage_bps=CFG["slippage_bps"], commission_bps=CFG["commission_bps"],
        borrow_bps_pa=CFG["borrow_bps_pa"], fx_basis_bps=CFG["fx_basis_bps"],
        use_risk_sizing=True, ewm_alpha=CFG["ewm_alpha"],
        target_vol=CFG["target_vol"], max_leverage=CFG["max_leverage"],
        gate=tmp["coint_on"],
        sd_beta=None, x_log=x_log,
        uncertainty_weight=CFG["uncertainty_weight"]
    )

    # Kalman (dinâmico)
    bt_kf, sum_kf = backtest_next_bar(
        tmp,
        z_col="z_kf",
        theo_col="bdr_teo_kf",
        z_entry=CFG["z_enter"], z_exit=CFG["z_exit"], z_stop=CFG["z_stop"],
        slippage_bps=CFG["slippage_bps"], commission_bps=CFG["commission_bps"],
        borrow_bps_pa=CFG["borrow_bps_pa"], fx_basis_bps=CFG["fx_basis_bps"],
        use_risk_sizing=True, ewm_alpha=CFG["ewm_alpha"],
        target_vol=CFG["target_vol"], max_leverage=CFG["max_leverage"],
        gate=tmp["gate_kf"],
        sd_beta=tmp["sd_beta_t"], x_log=x_log,
        uncertainty_weight=CFG["uncertainty_weight"]
    )

    # Kalman + HMM (se ligado)
    sum_kf_hmm = None
    if CFG.get("use_hmm_gate", False):
        bt_kf_hmm, sum_kf_hmm = backtest_next_bar(
            tmp,
            z_col="z_kf",
            theo_col="bdr_teo_kf",
            z_entry=CFG["z_enter"], z_exit=CFG["z_exit"], z_stop=CFG["z_stop"],
            slippage_bps=CFG["slippage_bps"], commission_bps=CFG["commission_bps"],
            borrow_bps_pa=CFG["borrow_bps_pa"], fx_basis_bps=CFG["fx_basis_bps"],
            use_risk_sizing=True, ewm_alpha=CFG["ewm_alpha"],
            target_vol=CFG["target_vol"], max_leverage=CFG["max_leverage"],
            gate=tmp["gate_kf_hmm"],
            sd_beta=tmp["sd_beta_t"], x_log=x_log,
            uncertainty_weight=CFG["uncertainty_weight"]
        )
        bt_kf_hmm_path = out_pairs_dir / f"{bdr}_{adr}_bt_kf_hmm.parquet"
        bt_kf_hmm.reset_index().rename(columns={'index':'datetime'}).to_parquet(bt_kf_hmm_path, index=False)

    # salvar curvas base e KF
    bt_base_path = out_pairs_dir / f"{bdr}_{adr}_bt_base.parquet"
    bt_kf_path   = out_pairs_dir / f"{bdr}_{adr}_bt_kf.parquet"
    bt_base.reset_index().rename(columns={'index':'datetime'}).to_parquet(bt_base_path, index=False)
    bt_kf.reset_index().rename(columns={'index':'datetime'}).to_parquet(bt_kf_path,   index=False)

    # resumo
    summary = {
        "bdr": bdr, "adr": adr,
        "overlap_obs": int(len(tmp)),
        "ratio_used": float(ratio_used),
        "ratio_method": ratio_method,
        "mae_teorico": float(mae_base),
        "mae_kf": float(mae_kf),
        "corr_log": float(diag["corr_log"]),
        "median_ratio_diag": float(diag["median_ratio"]),
        # backtests
        "sharpe_base": sum_base["sharpe_hourly_annualized"],
        "pnl_base": sum_base["pnl_total"],
        "mdd_base": sum_base["mdd"],
        "trades_base": sum_base["trades"],
        "sharpe_kf": sum_kf["sharpe_hourly_annualized"],
        "pnl_kf": sum_kf["pnl_total"],
        "mdd_kf": sum_kf["mdd"],
        "trades_kf": sum_kf["trades"],
    }
    if CFG.get("use_hmm_gate", False) and (sum_kf_hmm is not None):
        summary.update({
            "sharpe_kf_hmm": sum_kf_hmm["sharpe_hourly_annualized"],
            "pnl_kf_hmm":    sum_kf_hmm["pnl_total"],
            "mdd_kf_hmm":    sum_kf_hmm["mdd"],
            "trades_kf_hmm": sum_kf_hmm["trades"],
        })

    # logging
    base_msg = (f"[{bdr}_{adr}] BASE Sharpe={summary['sharpe_base']:.2f} "
                f"({summary['trades_base']} trades) | "
                f"KF Sharpe={summary['sharpe_kf']:.2f} "
                f"({summary['trades_kf']} trades) | overlap={summary['overlap_obs']}")
    if CFG.get("use_hmm_gate", False) and ("sharpe_kf_hmm" in summary):
        base_msg += f" | KF+HMM Sharpe={summary['sharpe_kf_hmm']:.2f} ({summary['trades_kf_hmm']} trades)"
    log.info(base_msg)

    return summary, out_path


# ============================ REPORTS / GRÁFICOS ============================

import os
from matplotlib.dates import DateFormatter

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _bars_per_day(index: pd.DatetimeIndex) -> float:
    if len(index) == 0: return 1.0
    counts = pd.Series(index.normalize()).value_counts()
    return float(np.median(counts.values))

def _rolling_sharpe(pnl: pd.Series, win: int = 63, index: pd.DatetimeIndex | None = None) -> pd.Series:
    """Sharpe rolling (janela 'win'), anualizado pela cadência média (barras/dia)."""
    pnl = pd.Series(pnl).dropna()
    if index is None: index = pnl.index
    ann = np.sqrt(max(_bars_per_day(index), 1.0) * 252.0)
    mu = pnl.rolling(win).mean()
    sd = pnl.rolling(win).std(ddof=1)
    rs = (mu / (sd + 1e-12)) * ann
    return rs.rename(f"roll_sharpe_{win}")

def _empirical_transition_matrix(mr_flag: pd.Series) -> np.ndarray:
    """Matriz 2x2 empírica a partir do flag de regime (0/1)."""
    s = mr_flag.fillna(False).astype(int)
    s_prev = s.shift(1)
    mask = s_prev.notna()
    s = s[mask]; s_prev = s_prev[mask].astype(int)
    cm = np.zeros((2,2), dtype=float)
    for i in (0,1):
        for j in (0,1):
            cm[i,j] = float(((s_prev==i) & (s==j)).sum())
    row_sums = cm.sum(axis=1, keepdims=True)
    row_sums[row_sums==0] = 1.0
    return cm / row_sums

def _trade_segments_from_pos(pos: pd.Series):
    """Decompõe série de posição em trades (in/out) retornando lista de (t_in, t_out)."""
    pos = pd.Series(pos).fillna(0.0)
    on = (pos != 0.0).astype(int)
    flips = on.diff().fillna(on).ne(0)
    idx_flips = list(on[flips].index)
    if not idx_flips:
        return []
    # garante que começamos em entrada
    if on.loc[idx_flips[0]] == 0 and len(idx_flips) > 1:
        idx_flips = idx_flips[1:]
    trades = []
    for k in range(0, len(idx_flips), 2):
        t_in = idx_flips[k]
        if k+1 < len(idx_flips):
            t_out = idx_flips[k+1]
        else:
            t_out = pos.index[-1]
        # trade válido tem pelo menos 1 passo
        if t_in < t_out:
            trades.append((t_in, t_out))
    return trades

def _mark_entries_exits_stops(pos: pd.Series, z: pd.Series, z_stop: float):
    """Detecta entradas/saídas e classifica saídas por stop (|z|>=z_stop) vs 'exit'."""
    pos = pd.Series(pos).fillna(0.0)
    z = pd.Series(z)
    state = (pos != 0.0).astype(int)
    d = state.diff().fillna(state)
    entries = list(state[d==+1].index)
    exits   = list(state[d==-1].index)
    stops, normal_exits = [], []
    for t in exits:
        if t in z.index and (abs(float(z.loc[t])) >= float(z_stop)):
            stops.append(t)
        else:
            normal_exits.append(t)
    return entries, normal_exits, stops

def _save_df_to_excel(writer, sheet_name: str, df: pd.DataFrame):
    df_out = df.copy()
    if 'datetime' not in df_out.columns and isinstance(df_out.index, pd.DatetimeIndex):
        df_out = df_out.reset_index().rename(columns={'index':'datetime'})
    df_out.to_excel(writer, sheet_name=sheet_name, index=False)

def make_pair_report(pair_parquet_path: Path, out_root: Path, bdr: str, adr: str, cfg: dict):
    """
    Lê os parquets gerados no pipeline e produz:
      - BDR vs Teórico (estático & KF) + beta_t / sd_beta_t (2º eixo)
      - Spread_kf & Z_kf com faixas de regime e marcações de trades
      - P(MR) do HMM + threshold (se existir p_mr)
      - Matriz de transição empírica (heatmap)
      - Equities comparadas (BASE, KF, KF+HMM) + drawdown
      - Distribuição de PnL por trade (estratificada)
      - Turnover & custos (KF vs KF+HMM)
      - Rolling Sharpe (63)
      - Sensibilidade do threshold do HMM (linha)
    Salva:
      - PNGs em ../data/output/reports/{BDR}_{ADR}/
      - Um Excel por par com uma aba por gráfico contendo *exatamente* os dados usados.
    """
    name = f"{bdr}_{adr}"
    report_dir = out_root / "reports" / name
    _ensure_dir(report_dir)

    # === Carrega o .parquet do par e os backtests ===
    df = pd.read_parquet(pair_parquet_path).set_index('datetime')
    df.index = pd.to_datetime(df.index)

    pairs_dir = pair_parquet_path.parent
    bt_base_path = pairs_dir / f"{name}_bt_base.parquet"
    bt_kf_path   = pairs_dir / f"{name}_bt_kf.parquet"
    bt_kf_hmm_path = pairs_dir / f"{name}_bt_kf_hmm.parquet"

    bt_base = pd.read_parquet(bt_base_path).set_index('datetime') if bt_base_path.exists() else None
    bt_kf   = pd.read_parquet(bt_kf_path).set_index('datetime')   if bt_kf_path.exists()   else None
    bt_hmm  = pd.read_parquet(bt_kf_hmm_path).set_index('datetime') if bt_kf_hmm_path.exists() else None

    # Excel writer
    xlsx_path = report_dir / f"{name}_data_for_plots.xlsx"
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:

        # ---------------- (1) BDR vs Teórico (estático & KF) ----------------
        d1 = pd.DataFrame({
            "close_bdr":  df["close_bdr"],
            "bdr_teo":    df["bdr_teo"],
            "bdr_teo_kf": df.get("bdr_teo_kf", pd.Series(index=df.index)),
            "beta_t":     df.get("beta_t", pd.Series(index=df.index)),
            "sd_beta_t":  df.get("sd_beta_t", pd.Series(index=df.index)),
        }).dropna(subset=["close_bdr","bdr_teo"]).copy()
        _save_df_to_excel(writer, "bdr_vs_teo", d1)

        fig, ax1 = plt.subplots(figsize=(12,5))
        d1[["close_bdr","bdr_teo","bdr_teo_kf"]].dropna().plot(ax=ax1)
        ax1.set_title(f"{name} — BDR vs Teórico (estático & KF)")
        ax1.set_xlabel("Tempo"); ax1.set_ylabel("Preço (BRL)")
        ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))

        ax2 = ax1.twinx()
        d_beta = d1[["beta_t","sd_beta_t"]].dropna()
        if not d_beta.empty:
            d_beta["beta_t"].plot(ax=ax2, linestyle="--")
            d_beta["sd_beta_t"].plot(ax=ax2, linestyle=":")
            ax2.set_ylabel("β_t / sd_β_t")
        fig.tight_layout(); fig.savefig(report_dir / f"{name}_bdr_vs_teo.png", dpi=160); plt.close(fig)

        # ---------------- (2) Spread_kf & Z_kf + faixas de regime + trades ----------------
        spread_kf_price = (df["close_bdr"] - df.get("bdr_teo_kf", df["bdr_teo"])).rename("spread_kf_price")
        z_kf = df.get("z_kf", pd.Series(index=df.index))
        gate_kf = df.get("gate_kf", pd.Series(False, index=df.index)).astype(bool)
        gate_kf_hmm = df.get("gate_kf_hmm", pd.Series(False, index=df.index)).astype(bool)

        # marcar entradas/saídas/stops a partir do bt_kf (se existir)
        entries, exits, stops = [], [], []
        if bt_kf is not None and "pos" in bt_kf.columns:
            entries, exits, stops = _mark_entries_exits_stops(bt_kf["pos"], z_kf, cfg["z_stop"])

        d2 = pd.DataFrame({
            "spread_kf_price": spread_kf_price,
            "z_kf": z_kf,
            "gate_kf": gate_kf,
            "gate_kf_hmm": gate_kf_hmm
        })
        _save_df_to_excel(writer, "spread_z_regimes", d2)

        fig, ax = plt.subplots(figsize=(12,6))
        d2["spread_kf_price"].dropna().plot(ax=ax)
        ax.set_title(f"{name} — Spread_kf & Z_kf com regimes")
        ax.set_xlabel("Tempo"); ax.set_ylabel("Spread (BRL)")
        ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))

        # faixas gate_kf (claro) e gate_kf_hmm (escuro)
        ylim = ax.get_ylim()
        on_kf = (gate_kf == True)
        on_hmm = (gate_kf_hmm == True)
        # blocos contínuos
        for on, alpha in [(on_kf, 0.15), (on_hmm, 0.3)]:
            runs = (on.astype(int).diff().fillna(on.astype(int)) != 0)
            idx = list(on.index[runs])
            if len(idx) == 0 and on.any():
                idx = [on.index[0]]
            if on.iloc[0]:
                idx = [on.index[0]] + idx
            # fecha blocos
            for i in range(0, len(idx), 2):
                start = idx[i]
                end = idx[i+1] if i+1 < len(idx) else on.index[-1]
                if on.loc[start]:
                    ax.axvspan(start, end, color="tab:green", alpha=alpha, lw=0)

        # eixo secundário para z_kf
        axz = ax.twinx()
        z_kf.dropna().plot(ax=axz, linestyle="--")
        axz.set_ylabel("Z_kf")
        axz.axhline(cfg["z_enter"], linestyle=":")
        axz.axhline(-cfg["z_enter"], linestyle=":")
        axz.axhline(cfg["z_exit"], linestyle="-.")
        axz.axhline(-cfg["z_exit"], linestyle="-.")
        axz.axhline(cfg["z_stop"], linestyle="--")
        axz.axhline(-cfg["z_stop"], linestyle="--")

        # marcação de entradas/saídas/stops
        for t in entries: ax.axvline(t, color="tab:blue", alpha=0.5, linewidth=1.0)
        for t in exits:   ax.axvline(t, color="tab:orange", alpha=0.7, linewidth=1.0)
        for t in stops:   ax.axvline(t, color="tab:red", alpha=0.9, linewidth=1.0)

        fig.tight_layout(); fig.savefig(report_dir / f"{name}_spread_z_regimes.png", dpi=160); plt.close(fig)

        # ---------------- (3) P(MR) e threshold (se existir) ----------------
        if "p_mr" in df.columns:
            thr = float(cfg["hmm_p_threshold"])
            d3 = pd.DataFrame({"p_mr": df["p_mr"], "threshold": pd.Series(thr, index=df.index)})
            _save_df_to_excel(writer, "p_mr", d3)

            fig, ax = plt.subplots(figsize=(12,4))
            d3["p_mr"].dropna().plot(ax=ax)
            ax.axhline(thr, linestyle="--")
            ax.set_title(f"{name} — P(MR) filtrada (sem look-ahead)"); ax.set_ylim(0, 1)
            ax.set_xlabel("Tempo"); ax.set_ylabel("P(MR)")
            ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
            # sombrear onde p_mr.shift(1) >= thr
            mr_ok = (df["p_mr"].shift(1) >= thr)
            for i, v in mr_ok.groupby((mr_ok != mr_ok.shift()).cumsum()):
                block = v[v]
                if not block.empty:
                    ax.axvspan(block.index[0], block.index[-1], color="tab:green", alpha=0.15, lw=0)
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_pmr.png", dpi=160); plt.close(fig)

            # --------- (4) Matriz de transição empírica (heatmap) ----------
            mr_flag = (df["p_mr"] > 0.5)  # definição simples de estado MR
            Aemp = _empirical_transition_matrix(mr_flag)
            # salva em aba
            A_df = pd.DataFrame(Aemp, index=["prev=NR","prev=MR"], columns=["NR","MR"])
            _save_df_to_excel(writer, "A_empirical", A_df.reset_index().rename(columns={'index':'prev'}))

            fig, ax = plt.subplots(figsize=(4,3))
            im = ax.imshow(Aemp, aspect="equal")
            ax.set_xticks([0,1]); ax.set_xticklabels(["NR","MR"])
            ax.set_yticks([0,1]); ax.set_yticklabels(["prev=NR","prev=MR"])
            for i in range(2):
                for j in range(2):
                    ax.text(j, i, f"{Aemp[i,j]:.2f}", ha="center", va="center", color="w")
            ax.set_title(f"{name} — Matriz de transição (empírica)")
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_A_empirical.png", dpi=160); plt.close(fig)

        # ---------------- (5) Equities comparadas + drawdown ----------------
        eq_df = pd.DataFrame(index=df.index)
        if bt_base is not None and "equity" in bt_base: eq_df["equity_base"] = bt_base["equity"]
        if bt_kf   is not None and "equity" in bt_kf:   eq_df["equity_kf"]   = bt_kf["equity"]
        if bt_hmm  is not None and "equity" in bt_hmm:  eq_df["equity_hmm"]  = bt_hmm["equity"]
        dd_df = eq_df.apply(lambda s: s - s.cummax())
        d5 = pd.concat([eq_df, dd_df.add_suffix("_dd")], axis=1).dropna(how="all")
        _save_df_to_excel(writer, "equities_dd", d5)

        if not d5.empty:
            fig, ax = plt.subplots(figsize=(12,6))
            eq_df.dropna(how="all").plot(ax=ax)
            ax.set_title(f"{name} — Equity (BASE vs KF vs KF+HMM)")
            ax.set_xlabel("Tempo"); ax.set_ylabel("P/L acumulado (unidades de preço)")
            ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_equities.png", dpi=160); plt.close(fig)

            fig, ax = plt.subplots(figsize=(12,3.5))
            dd_df.dropna(how="all").plot(ax=ax)
            ax.set_title(f"{name} — Drawdown")
            ax.set_xlabel("Tempo"); ax.set_ylabel("DD")
            ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_drawdown.png", dpi=160); plt.close(fig)

        # ---------------- (6) Distribuição PnL por trade -------------------
        trades_rows = []
        def _collect_trades(bt: pd.DataFrame, label: str):
            if bt is None or "equity" not in bt or "pos" not in bt: return
            segments = _trade_segments_from_pos(bt["pos"])
            for (t0, t1) in segments:
                eq = bt["equity"]
                try:
                    pnl = float(eq.loc[t1] - eq.loc[t0])
                except Exception:
                    continue
                trades_rows.append({"strategy": label, "t_in": t0, "t_out": t1, "pnl": pnl})

        _collect_trades(bt_kf, "KF")
        _collect_trades(bt_hmm, "KF+HMM")

        # bloqueados pelo HMM: trades em KF que não aparecem no KF+HMM (aprox por tempo)
        if bt_kf is not None and bt_hmm is not None:
            seg_kf  = _trade_segments_from_pos(bt_kf["pos"])
            seg_hmm = _trade_segments_from_pos(bt_hmm["pos"])
            hmm_intervals = pd.IntervalIndex.from_tuples([(a,b) for a,b in seg_hmm], closed="both") if seg_hmm else pd.IntervalIndex([])
            for (t0, t1) in seg_kf:
                if not any([(t0 in iv) or (t1 in iv) or ((iv.left<=t0) and (iv.right>=t1)) for iv in hmm_intervals]):
                    eq = bt_kf["equity"]
                    if t0 in eq.index and t1 in eq.index:
                        pnl = float(eq.loc[t1] - eq.loc[t0])
                        trades_rows.append({"strategy":"Bloqueado_HMM", "t_in": t0, "t_out": t1, "pnl": pnl})

        trades_df = pd.DataFrame(trades_rows)
        _save_df_to_excel(writer, "trades", trades_df)

        if not trades_df.empty:
            fig, ax = plt.subplots(figsize=(8,4))
            # histograma simples por estratégia
            for label, sub in trades_df.groupby("strategy"):
                ax.hist(sub["pnl"].values, bins=30, alpha=0.5, label=label)
            ax.set_title(f"{name} — PnL por trade (estratificado)")
            ax.set_xlabel("PnL/trade"); ax.set_ylabel("freq")
            ax.legend()
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_trades_hist.png", dpi=160); plt.close(fig)

        # ---------------- (7) Turnover & custos -----------------------------
        def _turnover(bt: pd.DataFrame):
            if bt is None or "pos" not in bt: return pd.Series(index=df.index, dtype=float)
            return bt["pos"].diff().abs().rename("turnover")

        d7 = pd.DataFrame({
            "turnover_kf": _turnover(bt_kf),
            "turnover_hmm": _turnover(bt_hmm),
            "trade_cost_kf": bt_kf["trade_cost"] if bt_kf is not None and "trade_cost" in bt_kf else pd.Series(index=df.index),
            "borrow_cost_kf": bt_kf["borrow_cost"] if bt_kf is not None and "borrow_cost" in bt_kf else pd.Series(index=df.index),
            "fx_cost_kf": bt_kf["fx_cost"] if bt_kf is not None and "fx_cost" in bt_kf else pd.Series(index=df.index),
            "trade_cost_hmm": bt_hmm["trade_cost"] if bt_hmm is not None and "trade_cost" in bt_hmm else pd.Series(index=df.index),
            "borrow_cost_hmm": bt_hmm["borrow_cost"] if bt_hmm is not None and "borrow_cost" in bt_hmm else pd.Series(index=df.index),
            "fx_cost_hmm": bt_hmm["fx_cost"] if bt_hmm is not None and "fx_cost" in bt_hmm else pd.Series(index=df.index),
        })
        _save_df_to_excel(writer, "turnover_costs", d7)

        if not d7.dropna(how="all").empty:
            fig, ax = plt.subplots(figsize=(12,4))
            d7[["turnover_kf","turnover_hmm"]].dropna(how="all").plot(ax=ax)
            ax.set_title(f"{name} — Turnover (KF vs KF+HMM)")
            ax.set_xlabel("Tempo"); ax.set_ylabel("|Δpos|")
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_turnover.png", dpi=160); plt.close(fig)

            # custos empilhados (soma diária)
            fig, ax = plt.subplots(figsize=(12,4))
            costs_kf = d7[["trade_cost_kf","borrow_cost_kf","fx_cost_kf"]].fillna(0.0).sum(axis=1)
            costs_hm = d7[["trade_cost_hmm","borrow_cost_hmm","fx_cost_hmm"]].fillna(0.0).sum(axis=1)
            pd.DataFrame({"KF":costs_kf, "KF+HMM":costs_hm}).plot(ax=ax)
            ax.set_title(f"{name} — Custos (diários)")
            ax.set_xlabel("Tempo"); ax.set_ylabel("Custo")
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_costs.png", dpi=160); plt.close(fig)

        # ---------------- (8) Rolling Sharpe (63) ---------------------------
        d8 = pd.DataFrame(index=df.index)
        if bt_base is not None and "pnl_net" in bt_base:
            d8["RS_base"] = _rolling_sharpe(bt_base["pnl_net"], 63, bt_base.index)
        if bt_kf is not None and "pnl_net" in bt_kf:
            d8["RS_kf"] = _rolling_sharpe(bt_kf["pnl_net"], 63, bt_kf.index)
        if bt_hmm is not None and "pnl_net" in bt_hmm:
            d8["RS_hmm"] = _rolling_sharpe(bt_hmm["pnl_net"], 63, bt_hmm.index)
        _save_df_to_excel(writer, "rolling_sharpe", d8)

        if not d8.dropna(how="all").empty:
            fig, ax = plt.subplots(figsize=(12,4))
            d8.dropna(how="all").plot(ax=ax)
            ax.set_title(f"{name} — Rolling Sharpe (63)")
            ax.set_xlabel("Tempo"); ax.set_ylabel("Sharpe anualizado")
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_rolling_sharpe.png", dpi=160); plt.close(fig)

        # ---------------- (9) Sensibilidade ao threshold do HMM ------------
        if "p_mr" in df.columns and bt_kf is not None:
            # roda backtests rápidos (reutiliza função original) variando o limiar
            thresholds = np.arange(0.50, 0.81, 0.05)
            results = []
            for thr in thresholds:
                gate = df.get("gate_kf", pd.Series(False, index=df.index)).astype(bool) & (df["p_mr"].shift(1) >= thr)
                # pequeno backtest só para Sharpe
                tmp_local = df.copy()
                z_col = "z_kf" if "z_kf" in df.columns else "z"
                tmp_local[z_col] = tmp_local[z_col].astype(float)
                bt_tmp, sum_tmp = backtest_next_bar(
                    tmp_local,
                    z_col=z_col,
                    theo_col="bdr_teo_kf" if "bdr_teo_kf" in df.columns else "bdr_teo",
                    z_entry=cfg["z_enter"], z_exit=cfg["z_exit"], z_stop=cfg["z_stop"],
                    slippage_bps=cfg["slippage_bps"], commission_bps=cfg["commission_bps"],
                    borrow_bps_pa=cfg["borrow_bps_pa"], fx_basis_bps=cfg["fx_basis_bps"],
                    use_risk_sizing=True, ewm_alpha=cfg["ewm_alpha"],
                    target_vol=cfg["target_vol"], max_leverage=cfg["max_leverage"],
                    gate=gate, sd_beta=df.get("sd_beta_t"), x_log=np.log(df["bdr_teo"].clip(lower=1e-12)),
                    uncertainty_weight=cfg["uncertainty_weight"]
                )
                results.append({"threshold": round(float(thr),2), "sharpe": sum_tmp["sharpe_hourly_annualized"]})
            sens_df = pd.DataFrame(results)
            _save_df_to_excel(writer, "sens_threshold", sens_df)

            fig, ax = plt.subplots(figsize=(6,4))
            ax.plot(sens_df["threshold"], sens_df["sharpe"], marker="o")
            ax.set_title(f"{name} — Sensibilidade do threshold do HMM")
            ax.set_xlabel("threshold"); ax.set_ylabel("Sharpe anualizado")
            fig.tight_layout(); fig.savefig(report_dir / f"{name}_sens_threshold.png", dpi=160); plt.close(fig)

        # ---------------- (10) Metadados/params -----------------------------
        meta = pd.DataFrame({
            "key": list(cfg.keys()),
            "value": [str(cfg[k]) for k in cfg.keys()]
        })
        _save_df_to_excel(writer, "params_cfg", meta)

    # Log final
    log.info(f"[reports] Gráficos e Excel salvos em: {report_dir}")
    return report_dir


# ============================ MAIN ============================

def main():
    OUT_DIR = Path(CFG["out_dir"])
    (OUT_DIR / "pairs").mkdir(parents=True, exist_ok=True)
    (OUT_DIR / "plots").mkdir(parents=True, exist_ok=True)

    # Carrega dados
    log.info("Lendo BDRs…")
    bdr_map = load_bdr_sheets(Path(CFG["path_bdr_xlsx"]))
    log.info(f"BDRs lidas: {len(bdr_map)}")

    log.info("Lendo ADRs…")
    us_map  = load_us_sheets(Path(CFG["path_us_xlsx"]))
    log.info(f"ADRs lidas: {len(us_map)}")

    log.info("Lendo FX (USDBRL)…")
    fx_df   = load_fx(Path(CFG["path_fx_xlsx"]))

    # Loop de pares
    rows_summary = []
    for bdr, adr in CFG["pairs"].items():
        if bdr not in bdr_map:
            log.warning(f"BDR '{bdr}' não encontrado; pulando.")
            continue
        if adr not in us_map:
            log.warning(f"ADR '{adr}' não encontrado; pulando.")
            continue

        res = run_pipeline_one_pair(
            bdr, adr, bdr_map, us_map, fx_df,
            out_pairs_dir=(OUT_DIR / "pairs"),
            plots_dir=(OUT_DIR / "plots")
        )
        if res is None:
            continue
        summary, pair_parquet_path = res
        rows_summary.append(summary)
        try:
            make_pair_report(pair_parquet_path, OUT_DIR, bdr, adr, CFG)
        except Exception as e:
           log.exception(f"Falha ao gerar report de {bdr}_{adr}: {e}")

    # Consolidado
    if rows_summary:
        df_sum = pd.DataFrame(rows_summary).sort_values("sharpe_kf", ascending=False)
        df_sum.to_csv(OUT_DIR / "summary_pairs.csv", index=False, float_format="%.8f")
        log.info("\n== Top 10 por Sharpe (KF) ==")
        cols = ["bdr","adr","overlap_obs","mae_teorico","mae_kf",
                "sharpe_base","pnl_base","trades_base",
                "sharpe_kf","pnl_kf","trades_kf","mdd_kf"]
        if CFG.get("use_hmm_gate", False):
            cols += ["sharpe_kf_hmm","pnl_kf_hmm","trades_kf_hmm","mdd_kf_hmm"]
        # imprime só colunas que existem (caso algum par não tenha HMM por falta de dados)
        cols = [c for c in cols if c in df_sum.columns]
        log.info("\n" + df_sum[cols].head(10).to_string(index=False))
        
        # Gráfico e Excel do ranking por par (por Sharpe KF / KF+HMM se existir)
        rank_dir = OUT_DIR / "reports" / "ranking"
        rank_dir.mkdir(parents=True, exist_ok=True)

        df_plot = df_sum.copy()
        metric = "sharpe_kf_hmm" if "sharpe_kf_hmm" in df_plot.columns else "sharpe_kf"
        df_plot = df_plot.sort_values(metric, ascending=True)

        # salva Excel com exatamente as colunas do gráfico
        df_plot_out = df_plot[["bdr","adr",metric,"mdd_kf" if metric=="sharpe_kf" else "mdd_kf_hmm"]].rename(
            columns={metric:"sharpe", "mdd_kf":"mdd", "mdd_kf_hmm":"mdd"}
        )
        df_plot_out.to_excel(rank_dir / "ranking_data_for_plot.xlsx", index=False)

        plt.figure(figsize=(8, max(4, len(df_plot_out)*0.4)))
        plt.barh(df_plot_out["bdr"]+"_"+df_plot_out["adr"], df_plot_out["sharpe"])
        plt.title(f"Ranking por par — {metric.upper()}")
        plt.xlabel("Sharpe anualizado"); plt.tight_layout()
        plt.savefig(rank_dir / "ranking_sharpe.png", dpi=160); plt.close()

    else:
        log.warning("Nenhum par válido processado.")

if __name__ == "__main__":
    main()
